<a id='top'></a>
# HANDBUILT IMAGE PROCESSING FUNCTIONS 
# Truncated version for spectral clustering

<div class="alert alert-block alert-success">
</div>

***
## Table of Contents

- [Reference To Constants & Predefined Kernels ](#reference-to-constants--predefined-kernels-)
- [Image Loading & Saving](#image-loading--saving)
- [Display](#display)
- [Statistics](#statistics)
- [Type Conversion](#type--conversion)
- [Algebraic Manipulations & General Enhancement](#algebraic-manipulations--general-enhancement)
- [Histograms & Data Processing](#histograms--data-processing)
- [Color](#color)
- [Noise Addition](#noise-addition)
- [Filtering (Denoising)](#filtering-denoising)
- [Filtering (Sharpening)](#filtering-sharpening)
- [Thresholding | Object Detection](#thresholding--object-detection)
- [Niblack, Bernsen, Sauvola](#niblack-bernsen-sauvola)
- [Convolution & Edge Detection](#convolution--edge-detection)
- [Morphology](#morphology)
- [Midterm General](#midterm-general)
- [Midterm Procedures](#midterm-procedures)
- [Feature Extraction](#feature-extraction)
- [Downsampling & Upsampling](#downsample-upsample)

[GO TO TOP](#top)

<a id="reference-to-constants--predefined-kernels-"></a>
### Reference To Constants & Predefined Kernels 
Note these can be accessed in file name: "Constants"

In [14]:
'''
#-------------------SOBEL------------------------
kernel3x3_sobel_x
kernel3x3_sobel_y
#-----------------PREWITT------------------------
kernel3x3_prewitt_x
kernel3x3_prewitt_y 
#-----------------ROBERTS------------------------
kernel2x2_roberts_x
kernel2x2_roberts_y
#-----------------LAPLACIAN------------------------
kernel3x3_laplacian1
kernel3x3_laplacian1I
kernel3x3_laplacian2
kernel7x7_laplacianDoG
#-----------------ROBINSON------------------------
kernel3x3_robinson_N
kernel3x3_robinson_NW
kernel3x3_robinson_W 
kernel3x3_robinson_SW
kernel3x3_robinson_S
kernel3x3_robinson_SE
kernel3x3_robinson_E
kernel3x3_robinson_NE

#-----------------CV2 SHAPES----------------------
                "RECT": cv2.MORPH_RECT,
                "CROSS": cv2.MORPH_CROSS,
                "ELLIPSE": cv2.MORPH_ELLIPSE,
                "HITMISS": cv2.MORPH_HITMISS,
'''

'\n#-------------------SOBEL------------------------\nkernel3x3_sobel_x\nkernel3x3_sobel_y\n#-----------------PREWITT------------------------\nkernel3x3_prewitt_x\nkernel3x3_prewitt_y \n#-----------------ROBERTS------------------------\nkernel2x2_roberts_x\nkernel2x2_roberts_y\n#-----------------LAPLACIAN------------------------\nkernel3x3_laplacian1\nkernel3x3_laplacian1I\nkernel3x3_laplacian2\nkernel7x7_laplacianDoG\n#-----------------ROBINSON------------------------\nkernel3x3_robinson_N\nkernel3x3_robinson_NW\nkernel3x3_robinson_W \nkernel3x3_robinson_SW\nkernel3x3_robinson_S\nkernel3x3_robinson_SE\nkernel3x3_robinson_E\nkernel3x3_robinson_NE\n\n#-----------------CV2 SHAPES----------------------\n                "RECT": cv2.MORPH_RECT,\n                "CROSS": cv2.MORPH_CROSS,\n                "ELLIPSE": cv2.MORPH_ELLIPSE,\n                "HITMISS": cv2.MORPH_HITMISS,\n'

In [2]:
''' Test two functions for speed comparison '''
'''
import timeit

# Load a numpy or PIL image to use for testing
image = image_name_here
# Measure the execution time of Code 1
code_1_time = timeit.timeit(lambda: function1(image), number=100)
# Measure the execution time of Code 2
code_2_time = timeit.timeit(lambda: function2(image), number=100)

print(f"Code 1 execution time: {code_1_time:.6f} seconds")
print(f"Code 2 execution time: {code_2_time:.6f} seconds")
'''

'\nimport timeit\n\n# Load a numpy or PIL image to use for testing\nimage = image_name_here\n# Measure the execution time of Code 1\ncode_1_time = timeit.timeit(lambda: function1(image), number=100)\n# Measure the execution time of Code 2\ncode_2_time = timeit.timeit(lambda: function2(image), number=100)\n\nprint(f"Code 1 execution time: {code_1_time:.6f} seconds")\nprint(f"Code 2 execution time: {code_2_time:.6f} seconds")\n'

[GO TO TOP](#top)

<a id="image-loading--saving"></a>
-----------------------------
## Image Loading & Saving

In [9]:
# for file_name in sorted(os.listdir(folder_path)):
def import_from_folder(folder_path, classification=None, title_addon=None, display_names=True, force_binary=False):
    im_np_gray_data = []
    im_np_gray_titles = []
    im_PIL_gray_data = []
    im_PIL_gray_titles = []
    labels = []
    
    if display_names and title_addon is not None:
        print(f"\n\nSINGLE CHANNEL VARIABLE NAMES OF LOADED {title_addon.upper()} IMAGES: \n")
    elif display_names:
        print("\n\nSINGLE CHANNEL VARIABLE NAMES OF LOADED IMAGES: \n")
        
    label = None
    if classification is not None:
        if classification == 'clean':
            label = 0
        elif classification == 'dirty':
            label = 1
        else:
            "WARNING: label exists but is unidentified. May interfere with Classification."
    
    filenames = os.listdir(folder_path)
    filenames.sort(key=lambda f: int(re.sub('\D', '', f)) if f.endswith(('.jpg', '.jpeg', '.png')) else float('inf'))
    
    for file_name in filenames:
        if file_name.endswith((".jpg", ".jpeg", ".png")):
            file_path = os.path.join(folder_path, file_name)
            im_PIL = Image.open(file_path)
            
            #if the image originates from an already thresholded image set, make sure it is loaded as such.
            if force_binary:
                im_PIL = single_threshold(im_PIL, 128)
            
            im_np = np.array(im_PIL)
            im_PIL_gray = im_PIL.convert("L")
            im_np_gray = np.array(im_PIL_gray)
            
            base_name = os.path.splitext(file_name)[0]
            if title_addon is None:
                im_PIL_var_name = f"{base_name}_PIL"
                im_np_var_name = f"{base_name}_np"
                im_PIL_gray_var_name = f"{base_name}_PIL_gray"
                im_np_gray_var_name = f"{base_name}_np_gray"
            else:
                im_PIL_var_name = f"{title_addon}_{base_name}_PIL"
                im_np_var_name = f"{title_addon}_{base_name}_np"
                im_PIL_gray_var_name = f"{title_addon}_{base_name}_PIL_gray"
                im_np_gray_var_name = f"{title_addon}_{base_name}_np_gray"
            
            if display_names:
                print(" - ", im_PIL_gray_var_name, "[+]", im_np_gray_var_name)
            
            im_np_gray_data.append(im_np_gray)
            im_np_gray_titles.append(im_np_gray_var_name)
            im_PIL_gray_data.append(im_PIL_gray)
            im_PIL_gray_titles.append(im_PIL_gray_var_name)
            if label is not None:
                labels.append((im_np_gray_var_name, label))
                
            globals()[im_PIL_var_name] = im_PIL
            globals()[im_np_var_name] = im_np
            globals()[im_PIL_gray_var_name] = im_PIL_gray
            globals()[im_np_gray_var_name] = im_np_gray

    return im_PIL_gray_data, im_PIL_gray_titles, labels

In [15]:
# def import_from_folder(folder_path, addon=None, display_names = True):
#     im_PIL_list = []
#     im_np_list = []
#     im_PIL_gray_list = []
#     im_np_gray_list = []
    
#     im_np_gray_data = []
#     im_np_gray_titles = []
    
#     im_PIL_gray_data = []
#     im_PIL_gray_titles = []
    
#     #print("\nVARIABLE NAMES OF LOADED", addon.upper(), "IMAGES: \n")
#     if display_names == True and addon != None:
#         print("\n\nSINGLE CHANNEL VARIABLE NAMES OF LOADED", addon.upper(), "IMAGES: \n")
#     elif display_names == True and addon == None:
#         print("\n\nSINGLE CHANNEL VARIABLE NAMES OF LOADED IMAGES: \n")
          
#     filenames = os.listdir(folder_path)
#     filenames.sort(key=lambda f: int(re.sub('\D', '', f)) if f.endswith(('.jpg', '.jpeg', '.png')) else float('inf'))
    
#     for file_name in filenames:
        
#         # Check if the file is an image
#         if file_name.endswith((".jpg", ".jpeg", ".png")):
            
#             # Read the image file and convert to numpy arrays
#             file_path = os.path.join(folder_path, file_name)
#             im_PIL = Image.open(file_path)
            
#             # ENSURE that binary files are binary
#             if addon == 'bin' or addon == 'provided':
#                 im_PIL = single_threshold(im_PIL, 128)
            
#             im_np = np.array(im_PIL)
#             im_PIL_gray = im_PIL.convert('L')
#             im_np_gray = np.array(im_PIL_gray)

#             # Extract the image name from the filename (no type extension)
#             base_name = os.path.splitext(file_name)[0]
            
#             if addon == None:
#                 # Create variable names using f-strings
#                 im_PIL_var_name = f"{base_name}_PIL"
#                 im_np_var_name = f"{base_name}_np"
#                 im_PIL_gray_var_name = f"{base_name}_PIL_gray"
#                 im_np_gray_var_name = f"{base_name}_np_gray"
#             else:
#                 # Create variable names using f-strings
#                 im_PIL_var_name = f"{addon}_{base_name}_PIL"
#                 im_np_var_name = f"{addon}_{base_name}_np"
#                 im_PIL_gray_var_name = f"{addon}_{base_name}_PIL_gray"
#                 im_np_gray_var_name = f"{addon}_{base_name}_np_gray"
            
#             # Display names of loaded files
#             if display_names == True:
#                 #print(" - ", im_PIL_var_name, "[+]", im_np_var_name, "[+]", im_PIL_gray_var_name, "[+]", im_np_gray_var_name)
#                 print(" - ", im_PIL_gray_var_name, "[+]", im_np_gray_var_name)
            
#             # Add the numpy arrays to the corresponding lists
#             im_PIL_list.append((im_PIL_var_name, im_PIL))
#             im_np_list.append((im_np_var_name, im_np))
#             im_PIL_gray_list.append((im_PIL_gray_var_name, im_PIL_gray))
#             im_np_gray_list.append((im_np_gray_var_name, im_np_gray))
            
#             # Create grey-only returnable lists for image data and image titles
#             im_np_gray_data.append(im_np_gray)
#             im_np_gray_titles.append(im_np_gray_var_name)
            
#             im_PIL_gray_data.append(im_PIL_gray)
#             im_PIL_gray_titles.append(im_PIL_gray_var_name)
            
#             # Assign the numpy arrays to separate variables
#             globals()[im_PIL_var_name] = im_PIL
#             globals()[im_np_var_name] = im_np
#             globals()[im_PIL_gray_var_name] = im_PIL_gray
#             globals()[im_np_gray_var_name] = im_np_gray

#     return im_PIL_gray_data, im_PIL_gray_titles   
#     #return im_np_gray_data, im_np_gray_titles       
#     #return im_PIL_list, im_np_list, im_PIL_gray_list, im_np_gray_list

In [97]:
def resize_images(images, width=512, height=512, display_status=True):
    
    #ensure that the image data is pil format
    images = numpy_to_pil(images)
    
    # Resize a single image
    if isinstance(images, Image.Image):
        if images.size[0] == width and images.size[1] == height:
            if display_status:
                print("resize requested: the single image did not need to be resized.")
            return images
        else:
            img_resized = images.resize((width, height))
            if display_status:
                print("resize requested: the single image has been resized.")
            return img_resized
    
    # Resize a list of images
    elif isinstance(images, list):
        resized_images = []
        for image in images:
            if image.size[0] == width and image.size[1] == height:
                resized_images.append(image)
                resized=False
            else:
                img_resized = image.resize((width, height))
                resized_images.append(img_resized)
                resized=True
        
        if display_status:
            if resized  == False:
                print("resize requested: the set of images did not need to be resized.")
            if resized  == True:
                print("resize requested: the set of images has been resized.")
                
        return resized_images
    
    else:
        raise TypeError("ERROR: Input image(s) could not be resized.")

In [43]:
# save SINGLE image
def save_image(image, filename, color_type='rgb'):
    """
    Detects the format of the image file and saves it to disk with the given filename.

    Parameters:
        image (str, numpy.ndarray, PIL.Image, cv2.Image): The input image file, as a 
        path to an image file, a numpy array, a PIL image, or a cv2 image.
        filename (str): The desired name of the output file, including the file extension.
        color_type (str): The color mode of the input image. Valid values are 'gray', 'rgb', 'hsl', or 'hsv'. Default is 'rgb'.
    """
    # Convert the input image to a PIL image
    if isinstance(image, str):
        with Image.open(image) as im:
            pil_image = im.copy()
    elif isinstance(image, np.ndarray):
        if len(image.shape) == 2:
            # Convert grayscale image to RGB for saving as PNG or JPEG
            pil_image = Image.fromarray(image).convert(color_type.upper())
        else:
            pil_image = Image.fromarray(image, mode=color_type.upper())
    elif isinstance(image, Image.Image):
        if image.mode == 'L':
            # Convert grayscale image to RGB for saving as PNG or JPEG
            pil_image = ImageOps.grayscale(image).convert(color_type.upper())
        else:
            pil_image = image.copy()
            if image.mode != color_type.upper():
                pil_image = pil_image.convert(color_type.upper())
    elif isinstance(image, cv2.UMat):
        pil_image = Image.fromarray(cv2.UMat.get(image))
    elif isinstance(image, cv2.Mat):
        if len(image.shape) == 2:
            # Convert grayscale image to RGB for saving as PNG or JPEG
            pil_image = Image.fromarray(image).convert(color_type.upper())
        else:
            pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), mode=color_type.upper())
    elif isinstance(image, cv2.VideoCapture):
        ret, frame = image.read()
        if frame.ndim == 2:
            # Convert grayscale image to RGB for saving as PNG or JPEG
            pil_image = Image.fromarray(frame).convert(color_type.upper())
        else:
            pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), mode=color_type.upper())
    else:
        raise ValueError(f'Invalid image type: {type(image)}')

    # Get the file format of the image
    file_format = pil_image.format

    # Save the image to disk with the given filename and format
    output_filename = filename + '.png'
    pil_image.save(output_filename)
    # with open(output_filename, 'wb') as f:
        # pil_image.save(f)

    print(f'Saved image to {output_filename}')

In [1]:
# save LIST of images
def save_images(images, filenames, color_type='rgb'):
    """
    Detects the format of the image file and saves it to disk with the given filename.

    Parameters:
        images (list): A list of input images files, as paths to image files, numpy arrays, PIL images, or cv2 images.
        filenames (list): A list of desired names of the output files, NOT including file extensions.
        color_type (str): The color mode of the input image. Valid values are 'gray', 'rgb', 'hsl', or 'hsv'. Default is 'rgb'.
    """
    for image, filename in zip(images, filenames):
        # Convert the input image to a PIL image
        if isinstance(image, str):
            with Image.open(image) as im:
                pil_image = im.copy()
        elif isinstance(image, np.ndarray):
            if len(image.shape) == 2:
                # Convert grayscale image to RGB for saving as PNG or JPEG
                pil_image = Image.fromarray(image).convert(color_type.upper())
            else:
                pil_image = Image.fromarray(image, mode=color_type.upper())
        elif isinstance(image, Image.Image):
            if image.mode == 'L':
                # Convert grayscale image to RGB for saving as PNG or JPEG
                pil_image = ImageOps.grayscale(image).convert(color_type.upper())
            else:
                pil_image = image.copy()
                if image.mode != color_type.upper():
                    pil_image = pil_image.convert(color_type.upper())
        elif isinstance(image, cv2.UMat):
            pil_image = Image.fromarray(cv2.UMat.get(image))
        elif isinstance(image, cv2.Mat):
            if len(image.shape) == 2:
                # Convert grayscale image to RGB for saving as PNG or JPEG
                pil_image = Image.fromarray(image).convert(color_type.upper())
            else:
                pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), mode=color_type.upper())
        elif isinstance(image, cv2.VideoCapture):
            ret, frame = image.read()
            if frame.ndim == 2:
                # Convert grayscale image to RGB for saving as PNG or JPEG
                pil_image = Image.fromarray(frame).convert(color_type.upper())
            else:
                pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), mode=color_type.upper())
        else:
            raise ValueError(f'Invalid image type: {type(image)}')

        # Get the file format of the image
        file_format = pil_image.format

        # Save the image to disk with the given filename and format
        output_filename = filename + '.png'
        pil_image.save(output_filename)
        # with open(output_filename, 'wb') as f:
            # pil_image.save(f)

        print(f'Saved image to {output_filename}')

[GO TO TOP](#top)

<a id="display"></a>
-----------------------------
## Display

In [19]:
#PIL or nparray
def plot_image(image, title = '', save = False):
    pylab.title(title, size=20), pylab.imshow(image)
    if save == True:
        pylab.imsave(title + '.png', image)
    pylab.axis('off')
    
#     plt.close()

In [20]:
#PIL or nparray
def display_image(image, title=None, n=None, indirect_call=False, save=False): 
    # retrieve plot colors from main file
    %store -r plt_color
    %store -r label_color
    %store -r title_color
    
    if indirect_call == False:
        if n is not None:
            fig = plt.figure(figsize=(n,n))
        else:
            fig = plt.figure()
            
    if title is not None:    
        _ = plt.title(title, color = title_color)
    _ = plt.set_cmap('gray')
    _ = plt.axis('off')
    _ = plt.imshow(image)
    
    if save == True:
        _ = fig.savefig(title + '.png', dpi = 300, transparent=True, bbox_inches="tight")
    
#     plt.close()

In [24]:
def display_contours(image, contours, title=None, save=False):
    # retrieve plot colors from main file
    %store -r plt_color
    %store -r label_color
    %store -r title_color
    
    fig, ax = plt.subplots()
    ax.imshow(image, cmap='gray')
    for contour in contours:
        ax.plot(contour[:,0, 0], contour[:,0, 1], color='red', linewidth=2)
    if title is not None:
        ax.set_title(title, color=title_color)
    ax.set_axis_off()

    if save:
        if title is None:
             title = 'contours'
        plt.savefig(title + '.png', dpi=300, transparent=True, bbox_inches="tight")
        
    plt.close()

In [ ]:
def generate_contour_image(image, contours, line=0.5, title=None, display=False, save=False):
    import matplotlib as mpl
    
    blank_base = np.zeros_like(image)
    
    with mpl.rc_context(rc={'backend': 'Agg'}):
        fig, ax = plt.subplots()
        fig.set_size_inches(image.shape[1] / fig.dpi, image.shape[0] / fig.dpi)
        ax.imshow(blank_base, cmap='gray')
        
        for contour in contours:
            ax.plot(contour[:, 0, 0], contour[:, 0, 1], color='white', linewidth=line)
            
        if title is not None:
            ax.set_title(title)
        
        ax.set_axis_off()
        ax.margins(0)
        fig.tight_layout(pad=0)
            
        # Save the plot as a PIL image
        buf = BytesIO()
        fig.savefig(buf, format='png', dpi=fig.dpi, transparent=True, bbox_inches="tight", pad_inches=0)
        buf.seek(0)
        contour_image = Image.open(buf).convert('L')
    
    # Threshold the image to get binary white contours
    threshold = 128
    contour_image = contour_image.point(lambda p: p > threshold and 255)
    
    if save:
        if title is None:
            title = 'contours'
        plt.savefig(title + '.png', dpi=300, transparent=True, bbox_inches="tight")
    
    plt.close(fig)  # Close the figure to avoid showing it in the notebook
    
    return contour_image

In [21]:
# def generate_contour_image(image, contours, line=0.5, title=None, display=False, save=False):
#     blank_base = np.zeros_like(image)
    
#     fig, ax = plt.subplots()
#     fig.set_size_inches(image.shape[1] / fig.dpi, image.shape[0] / fig.dpi)
#     ax.imshow(blank_base, cmap='gray')
    
#     for contour in contours:
#         ax.plot(contour[:, 0, 0], contour[:, 0, 1], color='white', linewidth=line)
        
#     if title is not None:
#         ax.set_title(title)
    
#     ax.set_axis_off()
#     ax.margins(0)
#     fig.tight_layout(pad=0)
        
#     # Save the plot as a PIL image
#     buf = BytesIO()
#     fig.savefig(buf, format='png', dpi=fig.dpi, transparent=True, bbox_inches="tight", pad_inches=0)
#     buf.seek(0)
#     contour_image = Image.open(buf).convert('L')
    
#     # Threshold the image to get binary white contours
#     threshold = 128
#     contour_image = contour_image.point(lambda p: p > threshold and 255)
    
#     if save:
#         if title is None:
#             title = 'contours'
#         plt.savefig(title + '.png', dpi=300, transparent=True, bbox_inches="tight")
    
#     plt.close(fig)  # Close the figure to avoid showing it in the notebook
    
#     return contour_image

In [47]:
def display_hist(image, title='', indirect_call=False, m=None, n=None, save=False):
    image = pil_to_numpy(image)
    
    # retrieve plot colors from main file
    %store -r plt_color
    %store -r label_color
    %store -r title_color
    
    if indirect_call == False:
        if m is not None:
            fig = plt.figure(figsize=(m,n))
        else:
            fig = plt.figure(figsize=(4,2))
        
    hist = image.histogram()
    _ = plt.title(title + " Histogram", color = title_color)
    _ = plt.bar(range(256), hist, color = plt_color)
    if save == True:
        _ = fig.savefig(title + '.png', dpi = 300, transparent=True)

In [30]:
#types: display_pair(PIL, string, bool)
def display_pair(image, title=None, auto_title=False, m=None, n=None, save=False):
    image = numpy_to_pil(image)
    # retrieve plot colors from main file
    %store -r plt_color
    %store -r label_color
    %store -r title_color
    
    #FIX SO THAT IT DISPLAYS THE IMAGE NAME
    if title is None and auto_title == True:
            title = f"{image}"
    
    with plt.rc_context({'axes.edgecolor':label_color, 'xtick.color':label_color, 'ytick.color':label_color}):
        if m is not None:
            fig = plt.figure(figsize=(m,n))
        else:
            fig = plt.figure(figsize=(8,2))


         # show image
        _ = fig.add_subplot(1,2,1)
        if title is not None:
            _ = plt.title(title, color= title_color)
        _ = plt.set_cmap('gray')
        _ = plt.axis('off')
        _ = plt.imshow(image)

        # show image histogram
        hist = image.histogram()
        _ = fig.add_subplot(1,2,2)
        if title is not None:
            _ = plt.title(title + ' Histogram', color= title_color)
        _ = plt.bar(range(256), hist, color = plt_color )

        fig.tight_layout()
        if save == True:
            _ = fig.savefig(title + '.png', dpi = 300, transparent=True)
        _ = plt.show()

In [29]:
def plot_4x5(images, titles=None, auto_title=False, fileName=None, save=False):
    num_images = len(images)
    num_rows = 4
    num_cols = 5
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, 8))
    
    if titles is None and auto_title == True:
        titles = [f"Image {i+1}" for i in range(num_images)]
#         titles = [os.path.basename(img_path) for img_path in images]

    for i, ax in enumerate(axes.flat):
        if i < num_images:
            ax.imshow(images[i], cmap='gray')
            if titles is not None:
                ax.set_title(titles[i], color='lightpink')
        ax.axis('off')

    # Hide remaining axes
    for i in range(num_images, num_rows*num_cols):
        axes.flat[i].set_visible(False)
    
    # Set layout and save
    fig.tight_layout()
    if save == True:
        if fileName == None:
            fileName = 'plot_4x5'
        fig.savefig(fileName + '.png', dpi=300, transparent=True)
    plt.show()

In [30]:
def plot_rowsxcols(images, num_rows, num_cols, titles=None, auto_title=False, fileName=None, size=None, save=False):
    
    # retrieve plot colors from main file
    %store -r plt_color
    %store -r label_color
    %store -r title_color
    
    num_images = len(images)
    num_plots = num_rows * num_cols
    
    if num_images > num_plots:
        raise ValueError(f"Number of images ({num_images}) is greater than the number of plots ({num_plots})")
    
    if size is None:
        fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, 8))
    else:
        fig, axes = plt.subplots(num_rows, num_cols, figsize=(size[0], size[1]))
    
    if titles is None and auto_title == True:
        titles = [f"Image {i+1}" for i in range(num_images)]
    
    # Flatten the axes array
    axes = np.array(axes).flatten()

    for i in range(num_plots):
        if i < num_images:
            ax = axes[i]
            ax.imshow(images[i], cmap='gray')
            if titles is not None:
                ax.set_title(titles[i], color=title_color)
            ax.axis('off')
        else:
            # Hide remaining axes
            axes[i].set_visible(False)

    # Set labels for the axes
    plt.xlabel('eigenvectors')
    plt.ylabel('clusters')
    
    # Set layout and save
    fig.tight_layout()
    if save == True:
        if fileName == None:
            fileName = 'plot_images'
        fig.savefig(fileName + '.png', dpi=300, transparent=True)
    plt.show()
    
  
def plot_rowsxcols_style2(images, num_rows, num_cols, title=None, fileName=None, size=None, save=False):  
    num_images = len(images)
    num_plots = num_rows * num_cols
    
    if num_images > num_plots:
        raise ValueError(f"Number of images ({num_images}) is greater than the number of plots ({num_plots})")
    
    if size is None:
        fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, 8), sharex='col', sharey='row')
    else:
        fig, axes = plt.subplots(num_rows, num_cols, figsize=(size[0], size[1]), sharex='col', sharey='row')
    plt.suptitle(f'Clustering Results for {title}')
        
    # Set Individual Plot Labels
    for i in range(num_cols):
        plt.setp(axes[-1, i], xlabel=f'{i+2}')
    for i in range(num_rows):
        ylabels = plt.setp(axes[i:, 0], ylabel=f'{i+2}')
        for ylabel in ylabels:
            ylabel.set_rotation(0)
            
    # Flatten the axes array
    axes = np.array(axes).flatten()

    # Hide the subplot borders
    for ax in axes:
        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)
        ax.spines["bottom"].set_visible(False)
        ax.spines["left"].set_visible(False)

    for i in range(num_plots):
#         plt.setp(axes[-1:i], xlabel='x axis label', ylabel='y axis label')        
        if i < num_images:
            ax = axes[i]
            ax.imshow(images[i], cmap='gray')
            ax.grid(False)
            ax.xaxis.set_major_locator(plt.NullLocator())
            ax.yaxis.set_major_locator(plt.NullLocator())
        else:
            # Hide remaining axes
            axes[i].set_visible(False)

    # Set labels for the axes
    fig.supxlabel('eigenvectors')
    fig.supylabel('clusters')
    
    # Set layout and save
    fig.tight_layout()
    if save == True:
        if fileName == None:
            fileName = 'clustering_results'
        fig.savefig(f'{title}_clustering_results' + '.png', dpi=300, transparent=False)
    plt.show()
    
#plot images with predefined number of rows & columns
def plot_rowsxcols_style3(images, num_rows, num_cols, titles=None, auto_title=False, fileName=None, size=None, save=False):
    
    # retrieve plot colors from main file 
    num_images = len(images)
    num_plots = num_rows * num_cols
    
    if num_images > num_plots:
        raise ValueError(f"Number of images ({num_images}) is greater than the number of plots ({num_plots})")
    
    if size is None:
        fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, 8))
    else:
        fig, axes = plt.subplots(num_rows, num_cols, figsize=(size[0], size[1]))
    
    if titles is None and auto_title == True:
        titles = [f"Image {i+1}" for i in range(num_images)]
    
    for i, ax in enumerate(axes.flat):
        if i < num_images:
            ax.imshow(images[i], cmap='gray')
            if titles is not None:
                ax.set_title(titles[i])
        ax.axis('off')

    # Hide remaining axes
    for i in range(num_images, num_plots):
        axes.flat[i].set_visible(False)
    
    # Set layout and save
    fig.tight_layout()
    if save == True:
        if fileName == None:
            fileName = 'plot_images'
        fig.savefig(fileName + '.png', dpi=300, transparent=True)
    plt.show()


In [1]:
#plot images with auto-determined square dimensions
def plot_nxn(images, titles=None, auto_title=False, fileName=None, size=None, save=False):
    
    # retrieve plot colors from main file
    %store -r plt_color
    %store -r label_color
    %store -r title_color
    
    #if images is list or set of images...
    if isinstance(images, (list, tuple, np.ndarray)):
        num_images = len(images)
        num_plots = math.ceil(math.sqrt(num_images))

        if num_plots**2 == num_images:
            num_rows = num_cols = num_plots
        else:
            num_cols = num_plots
            num_rows = math.ceil(num_images / num_cols)

        if size is None:
            fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, 8))
        else:
            fig, axes = plt.subplots(num_rows, num_cols, figsize=(size[0], size[1]))
            
        if titles is None and auto_title == True:
            titles = [f"Image {i+1}" for i in range(num_images)]
    #         titles = [os.path.basename(img_path) for img_path in images]

        for i, ax in enumerate(axes.flat):
            if i < num_images:
                ax.imshow(images[i], cmap='gray')
                if titles is not None:
                    ax.set_title(titles[i], color = title_color)
            ax.axis('off')

        # Hide remaining axes
        for i in range(num_images, num_rows*num_cols):
            axes.flat[i].set_visible(False)
         
        # Set layout and save
        fig.tight_layout()
        if save == True:
            if fileName == None:
                fileName = 'plot_images'
            fig.savefig(fileName + '.png', dpi=300, transparent=True)
        plt.show()
    
    #if images is a single image...
    else:
        if titles is None: 
            display_image(images, indirect_call=True, save=save)
        else:
            display_image(images, titles[0], indirect_call=True, save=save)
        plt.show()


In [43]:
def plot_nxhist(images, titles=None, auto_title=False, fileName=None, save=False):
    
    # if data is a numpy array, convert it to PIL
    images = numpy_to_pil(images)
    
    # retrieve plot colors from main file
    %store -r plt_color
    %store -r label_color
    %store -r title_color

    #if images is list or set of images...
    if isinstance(images, (list, tuple, np.ndarray)):
        num_images = len(images)

        if titles is None and auto_title == True:
            titles = [f"Image {i+1}" for i in range(num_images)]

        with plt.rc_context({'axes.edgecolor':label_color, 'xtick.color':label_color, 'ytick.color':label_color}): 
            fig, axs = plt.subplots(num_images, 2, figsize=(8,num_images*2))

            for i in range(num_images):
                # show image
                if titles is not None:
                    axs[i, 0].set_title(titles[i], color=title_color)
                axs[i, 0].imshow(images[i], cmap='gray')
                axs[i, 0].axis('off')

                # show histogram
                hist = images[i].histogram()
                if titles is not None:
                    axs[i, 1].set_title(titles[i] + ' Histogram', color=title_color)
                axs[i, 1].bar(range(256), hist, color=plt_color)
                axs[i, 1].set_xlim([0, 256])

            fig.tight_layout()
            if save:
                _ = fig.savefig(fileName + '.png', dpi=300, transparent=True)
            _ = plt.show()
            
    #if images is a single image...        
    else:
        if titles is None:
            display_pair(images, save=save)
        else:
            display_pair(images, titles[0], save=save)
        plt.show()

In [ ]:
# def plot_nxhist(images, titles=None, auto_title=False, fileName=None, save=False):
    
#     # if data is a numpy array, convert it to PIL
#     images = numpy_to_pil(images)
    
#     # retrieve plot colors from main file
#     %store -r plt_color
#     %store -r label_color
#     %store -r title_color

#     #if images is list or set of images...
#     if isinstance(images, (list, tuple, np.ndarray)):
#         num_images = len(images)
#         num_rows = num_images * 2

#         if titles is None and auto_title == True:
#             titles = [f"Image {i+1}" for i in range(num_images)]

#         with plt.rc_context({'axes.edgecolor':label_color, 'xtick.color':label_color, 'ytick.color':label_color}): 
#             fig = plt.figure(figsize=(6,num_images*2))

#             for i in range(num_images):
#                 # show image
#                 _ = fig.add_subplot(num_rows, 2, i*2+1)
#                 if titles is not None:
#                     _ = plt.title(titles[i], color=title_color)
#                 _ = plt.set_cmap('gray')
#                 _ = plt.axis('off')
#                 _ = plt.imshow(images[i])

#                 # show histogram
#                 hist = images[i].histogram()
#                 _ = fig.add_subplot(num_rows, 2, i*2+2)
#                 if titles is not None:
#                     _ = plt.title(titles[i] + ' Histogram', color=title_color)
#                 _ = plt.bar(range(256), hist, color=plt_color)

#             fig.tight_layout()
#             if save:
#                 _ = fig.savefig(fileName + '.png', dpi=300, transparent=True)
#             _ = plt.show()
            
#     #if images is a single image...        
#     else:
#         display_pair(images, titles, m=6, n=2, save=save)


[GO TO TOP](#top)

<a id="statistics"></a>
-----------------------------
## Statistics

In [19]:
def compute_first_order_statistics(hist, display=False):
    """
    This function computes the first order statistics of a given histogram
    Parameters:
    hist (numpy.ndarray): A 1D numpy array representing the histogram.

    Returns:
    tuple: A tuple containing the following first order statistics:
    - n_pixels (int): The total number of pixels in the histogram.
    - min_val (int): The minimum value in the histogram.
    - max_val (int): The maximum value in the histogram.
    - mean (float): The mean value of the histogram.
    - std (float): The standard deviation of the histogram.
    - median (float): The median value of the histogram.
    - mode (int): The mode value of the histogram.
    """
    n_pixels = np.sum(hist)
    min_val = np.min(np.where(hist > 0))
    max_val = np.max(np.where(hist > 0))
    mean = np.sum(np.arange(len(hist)) * hist) / n_pixels
    std = np.sqrt(np.sum(((np.arange(len(hist)) - mean) ** 2) * hist) / n_pixels)
    median = np.median(np.where(hist > 0))
    mode = np.argmax(hist)
    
    if display:
        print(f"Number of pixels:", n_pixels)
        print(f"Minimum value:", min_val)
        print(f"Maximum value:", max_val)
        print(f"Mean value:", mean)
        print(f"Standard deviation:", std)
        print(f"Median value:", median)
        print(f"Mode value:", mode)
        
    return n_pixels, min_val, max_val, mean, std, median, mode

In [33]:
def calculate_data_distance(input1, input2, data_mode='image', measure='euclidean', display_hist=False, display=False):
    """
    Calculate the distance between two datasets (images or features) using a specified distance measure.
    
    Args:
        input1: First dataset (single image or feature array)
        input2: Second dataset (single image or feature array)
        data_mode (str): Type of data, either 'image' or 'feature'
        measure (str): Distance measure to use, either 'euclidean', 'linear', 'chi2', or 'cosine'
        display_hist (bool): Whether to display histogram data (only for image data_mode)
        display (bool): Whether to display the calculated distance

    Returns:
        float: The calculated distance between input1 and input2
    """
    distance = None
    if data_mode == 'image':
        if isinstance(input1, np.ndarray) or isinstance(input2, np.ndarray):
            print(f"Error: data_mode is '{data_mode}' but input is stored in ndarray")
            return -1      
        A = np.array(input1.histogram(), dtype='int64')
        B = np.array(input2.histogram(), dtype='int64')
        #display histogram data if desired
        if display_hist:
            print("Histogram1: \n", A.reshape(1,-1), "\nHistogram2: \n", B.reshape(1,-1))     
    elif data_mode == 'feature':
        if isinstance(input1, Image.Image) or isinstance(input2, Image.Image):
            print(f"Error: data_mode is '{data_mode}' but input is stored as PIL Image")
            return -1
        A = np.array(input1).astype(float)
        B = np.array(input2).astype(float)
        
    else:
        raise TypeError("Unrecognized data_mode. Currently accepting only: \'image\' and \'feature\'")
        
    #if the datasets can't be matched send warning
    if len(A) != len(B):
        raise ValueError("Datasets must have the same length in order to evaluate distance.")
        
    if measure == 'euclidean': 
        distance = 0
        distance = np.sqrt(np.sum((A - B)**2))
    elif measure == 'linear':
        distance = 0
        distance = np.mean(np.abs(A - B))  
    elif measure == 'chi2':
        distance = chi2_distance(A, B)
    elif measure == 'cosine':
        distance = 1 - np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))
    else:
        print("No Distance Calculation Method Specified")

    if distance is None:
        raise ValueError("No distance calculation method was specified")
    if display:
        print(f"\n{data_mode} {measure} distance = ", distance)
        
    return distance

In [1]:
# Given two matrices as input, the following function computes the minimum squared error(MSE) of the two matrices.
def calculate_mean_sq_error(mat1, mat2):
    min_sq = 0
    M, N = mat1.shape[:2]
    for i in range(M):
        for j in range(N):
            min_sq += np.square(mat1[i][j] - mat2[i][j])
    return min_sq

In [ ]:
# def intersection_distance(A, B):
#     return np.sum(np.minimum(A, B))

# def bhattacharyya_distance(A, B):
#     return -np.log(np.sum(np.sqrt(A * B)) / (np.sqrt(np.sum(A)) * np.sqrt(np.sum(B))))

# def calculate_data_distance(input1, input2, data_mode='image', measure='euclidean', display_hist=False, display=False):
#     # ... (previous code)
        
#     if measure == 'euclidean': 
#         distance = np.sqrt(np.sum((A - B)**2))
#     elif measure == 'linear':
#         distance = np.mean(np.abs(A - B))  
#     elif measure == 'chi2':
#         distance = chi2_distance(A, B)
#     elif measure == 'cosine':
#         distance = 1 - np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))
#     elif measure == 'intersection':
#         distance = intersection_distance(A, B)
#     elif measure == 'bhattacharyya':
#         distance = bhattacharyya_distance(A, B)
#     elif measure == 'emd':
#         from scipy.stats import wasserstein_distance
#         distance = wasserstein_distance(A, B)

In [15]:
def count_white_pixels(image):
    image = pil_to_numpy(image)
    height, width = image.shape

    white_pixel_count = 0
    
    # Loop through each pixel in the image
    for x in range(width):
        for y in range(height):
            # Get pixel color
            pixel = image[y, x]

            # Check if pixel is white (255) for RGB color channels
            if np.all(pixel == 255):
                # Increment counter
                white_pixel_count += 1

    return white_pixel_count

In [18]:
def calculate_perimeter(image, display=False, save=False):
    image = pil_to_numpy(image)
    
    contours, hierarchy = cv2.findContours(image.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    perimeter = sum(cv2.arcLength(contour, True) for contour in contours)
    extracted_contours = generate_contour_image(image, contours, title=None, save=save)

    perimeter = count_white_pixels(extracted_contours)
    perimeter_feat = np.array([perimeter])
    
    if display: 
        display_contours(image, contours, title=None, save=save)
        print("\nOriginal Image Shape = ", segmented_data[72].size)
        print("Generated Image Shape = ", extracted_contours.size)
        print("PERIMITER = ", perimeter)
    
    return perimeter_feat

In [ ]:
# Function to calculate Chi-distance
def chi2_distance(A, B, normalize=True):
    A = pil_to_numpy(A).astype(float)
    B = pil_to_numpy(B).astype(float)
    
    if normalize:
        A /= np.sum(A)
        B /= np.sum(B)
        
    # compute the chi-squared distance using above formula
    chi = 0.5 * np.sum([((a - b) ** 2) / (a + b + 1e-10) for (a, b) in zip(A, B)])
    
    return chi

In [1]:
def chi_squared_onimages(images, reference, display=False, save=False):
    chi_squared_vals = []
    diff = 0
    total = 0
    for i in range(len(images)):
        img = np.array(images[i]).astype(np.uint8)
        ref = np.array(reference).astype(np.uint8)
        diff = 0.5 * np.sum((img-ref)**2 / (img + ref + 1e-10))
        if display == True:
            print(diff)
        total += diff
        chi_squared_vals.append(diff)
    
    if display == True:   
        print("avg", total/len(chi_squared_vals))
    return chi_squared_vals

In [20]:
def chi_squared_onfeatures(feature_set, feature_benchmark):
    # calculate distances between all new_images_features and the single benchmark_image_features
    distances = []
    feature_benchmark = np.array(feature_benchmark)
    
    for i in range(len(feature_set)):
        feature_image = np.array(feature_set[i])
        dist = 0.5 * np.sum((feature_image - feature_benchmark) ** 2 / (feature_set[i] + feature_benchmark + 1e-10))
        distances.append(dist)

    return distances

[GO TO TOP](#top)

<a id="type--conversion"></a>
-----------------------------
## Type Conversion

In [22]:
def detect_gray_or_binary(image, force_binary = False):
    """
    This function detects if an image is binary or grayscale and optionally forces it to be binary.

    Parameters:
    image (numpy.ndarray): A 2D numpy array representing the image to check.
    force_binary (bool): A boolean indicating whether to force the image to be binary (default False).

    Returns:
     PIL.Image.Image: An instance of PIL.Image.Image representing the image after optional binarization.
    """
    image = pil_to_numpy(image)
    grayscale = False #assume false until non-binary value found
    for i in range(len(image)):
        for j in range(len(image[i])):
            pixelval = image[i][j]
            if (pixelval > 0) and (pixelval < 255):
                grayscale = True

    print("Grayscale Status: ", grayscale, "\nValues:\n", image)

    # Display and threshold if needed 
    display_image(image)
    if force_binary == True and grayscale == True:
        image = np.array(single_threshold(image, 128))
        display_image(image)

        grayscale = False #assume false until non-binary value found
        for i in range(len(image)):
            for j in range(len(image[i])):
                pixelval = image[i][j]
                if (pixelval > 0) and (pixelval < 255):
                    grayscale = True

        print("Grayscale Status UPDATED: ", grayscale, "\nValues:\n", grey_lap1_sigma1_np_gray)
        
    image = numpy_to_pil(image)
    return image

In [23]:
#works for both single images and sets of images
def detect_type(images, display_type = False):
    
    # Input is a list or other iterable of images. check type of first image
    if isinstance(images, (list, tuple)):
        first_image = images[0]
        if isinstance(first_image, np.ndarray):
            if display_type == True: 
                print("NumPy: Image SET contains images stored as NumPy arrays.")
            return 'numpy_set'
        elif isinstance(first_image, Image.Image):
            if display_type == True:
                print("PIL: Image SET contains images stored as PIL images.")
            return 'pil_set'
        else:
            raise TypeError("Image SET type not recognized.")
    
    # Input is a single image
    elif isinstance(images, np.ndarray):
        if display_type == True: 
            print("NumPy: Image SINGLE is stored as a NumPy array.")
        return 'numpy_single'
    elif isinstance(images, Image.Image):
        if display_type == True:
            print("PIL: Image SINGLE is stored as a PIL image.")
        return 'pil_single'
    else:
        raise TypeError("Image SET type not recognized.")

In [24]:
def pil_to_numpy(images):
    '''note: this function will not harm image(s) that are already in numpy format'''
    current_type = detect_type(images)
        
    # Immediately return numpy images
    if current_type == 'numpy_set' or current_type == 'numpy_single':
        return images
    
    # Convert single PIL image to NumPy array
    elif current_type == 'pil_single':
        numpy_img = np.array(images).astype(np.uint8)
        return numpy_img
    
    # Convert set of PIL images to set of NumPy arrays
    elif current_type == 'pil_set':
        numpy_images = []
        for img in images:
            numpy_img = np.array(img).astype(np.uint8)
            numpy_images.append(numpy_img)
        return numpy_images
    
    else:
        print("Attempted pil_to_numpy. Image type could not be identified -- data returned unmodified.")
        return images

In [25]:
def numpy_to_pil(images):
    '''note: this function will not harm image(s) that are already in pil format'''
    current_type = detect_type(images)
    
    # Immediately return PIL images
    if current_type == 'pil_single' or current_type == 'pil_set':
        return images
    
    # Convert single NumPy array to PIL image
    elif current_type == 'numpy_single':
        pil_img = Image.fromarray(images.astype(np.uint8))
        return pil_img
          
    # Convert set of NumPy arrays to set of PIL images
    elif current_type == 'numpy_set':
        pil_images = []
        for img in images:
            pil_img = Image.fromarray(img.astype(np.uint8))
            pil_images.append(pil_img)
        return pil_images

    else:
        print("Attempted numpy_to_pil. Image type could not be identified -- data returned unmodified.")
        return images


[GO TO TOP](#top)

<a id="algebraic-manipulations--general-enhancement"></a>
-----------------------------
## Algebraic Manipulations & General Enhancement

[GO TO TOP](#top)

<a id="histograms--data-processing"></a>
-----------------------------
## Histograms & Data Processing

In [68]:
def histogram_stretch(image):
    image = pil_to_numpy(image)    # if data is PIL, convert it to NumPy
    # np_image = np.array(image).astype(np.uint8)
    
    # Transformation to obtain stretching

    # constant = (255-0)/(np_image.max()-np_image.min())
    # img_stretch = np_image * constant
    # img_PIL_stretch = Image.fromarray(image_stretch)
    img_max = np_image.max()
    img_min = np_image.min()
    img_stretched = 255*((np_image - img_min)/(img_max - img_min)) #apply stretch
    
    img_stretched = numpy_to_pil(img_stretched)
    
    return img_stretched

In [51]:
# def histogram_stretch_clip(image, clip_percent=0.0):
#     np_image = np.array(image).astype(np.uint8)
    
#     if clip_percent > 1:
#         print("please enter the percent as a decimal value. Exe: 0.1 clips 10% of image boundary")
        
#     # Calculate the histogram of the image
#     hist, _ = np.histogram(np_image, bins=256, range=(0, 255))
#     # Calculate the cumulative sum of the histogram
#     cumsum = np.cumsum(hist)

#     # Determine the number of pixels to clip
#     num_pixels = int(clip_percent * np_image.size)
    
#     # Find the threshold for clipping the top 10% of the histogram
#     threshold = np.argmax(cumsum >= cumsum[-1] - num_pixels)

#     # Perform histogram stretching
#     img_min = np.percentile(np_image, clip_percent*100)
#     img_max = threshold
#     np_image = (np_image - img_min) * 255 / (img_max - img_min)
#     np_image[np_image > 255] = 255
#     np_image = np.clip(np_image, 0, 255)

#     img_PIL_stretch = Image.fromarray(np_image.astype(np.uint8))
#     return img_PIL_stretch

In [56]:
def histogram_stretch_clip(image, clip_percent=0.0):
    np_image = pil_to_numpy(image)    # if data is PIL, convert it to NumPy
    # np_image = np.array(image).astype(np.uint8)
    
    if clip_percent > 1:
        print("please enter the percent as a decimal value. Exe: 0.1 clips 10% of image boundary")
        
    # Calculate the histogram of the image
    hist, _ = np.histogram(np_image, bins=256, range=(0, 255))
    # Calculate the cumulative sum of the histogram
    cumsum = np.cumsum(hist)

    # Determine the number of pixels to clip
    num_pixels = int(clip_percent * np_image.size)
    
    # Find the threshold for clipping the top 10% of the histogram
    threshold = np.argmax(cumsum >= cumsum[-1] - num_pixels)

    # Perform histogram stretching
    img_min = np.percentile(np_image, clip_percent*100)
    img_max = threshold
    np_image = (np_image - img_min) * 255 / (img_max - img_min)
#     np_image[np_image > 255] = 255
    np_image = np.clip(np_image, 0, 255)
    
    img_stretch_clip = numpy_to_pil(np_image)
    
    return img_stretch_clip

In [24]:
def hist_matching(image, c, c_t):
    np_image = np.array(image).astype(np.uint8)

    pixels = np.arange(256)
    # find closest pixel-matches corresponding to the CDF of the input image, given the value of the CDF H of
    # the template image at the corresponding pixels, s.t. c_t = H(pixels) <=> pixels = H-1(c_t)
    new_pixels = np.interp(c, c_t, pixels)
    im = (np.reshape(new_pixels[np_image.ravel()], np_image.shape)).astype(np.uint8)
    
    im_PIL = Image.fromarray(im)
    
    return im

In [1]:
''' uses numpy histogram '''
''' 2nd fastest 0.144'''
def cdf(image, normalize=True):
    image = pil_to_numpy(image).astype(np.uint8)

    # Calculate the histogram of the image
    hist, _ = np.histogram(image.flatten(), bins=256, range=(0, 256))

    # Calculate cumulative distribution
    cdf = np.zeros_like(hist, dtype=np.float64)
    cumsum = 0
    for i, h in enumerate(hist):
        cumsum += h
        cdf[i] = cumsum

    # Normalize the CDF 
    if normalize:
        cdf = cdf / cdf[-1]

    return cdf

In [ ]:
''' Uses numpy histogram and cumsum '''
''' 1st fastest 0.128'''
def cdf_fast(image, normalize=True):
    image = pil_to_numpy(image).astype(np.uint8)

    # Calculate histogram of the image
    hist, _ = np.histogram(image.flatten(), bins=256, range=(0, 256))

    # Calculate cumulative distribution
    cdf = np.cumsum(hist).astype(np.float64)
    if normalize:
        cdf = cdf / cdf[-1]

    return cdf

In [4]:
# def image_distance(image1, image2):
#     hist1 = np.array(image1.histogram(), dtype='int64')
#     hist2 = np.array(image2.histogram(), dtype='int64')

#     # Calculate the distance between the two histograms
#     #distance = np.sum(((hist1 - hist2)**2)/(np.maximum(hist1, hist2)))
#     distance = euclidean_distances(hist1.reshape(1,-1), hist2.reshape(1,-1))
    
#     #print("Histogram1: \n", hist1.reshape(1,-1), "\nHistogram2: \n", hist2.reshape(1,-1))
#     #print("\nDistance = ", distance[0][0])
#     print(distance[0][0])
    
#     return distance

In [117]:
def image_distance(image1, image2, type='euclidean', display_hist=False):
    hist1 = np.array(image1.histogram(), dtype='int64')
    hist2 = np.array(image2.histogram(), dtype='int64')
    
    #display histogram data if desired
    if display_hist == True:
        print("Histogram1: \n", hist1.reshape(1,-1), "\nHistogram2: \n", hist2.reshape(1,-1))
        
    #if the histograms can't be matched send warning
    if len(hist1) != len(hist2):
        raise ValueError("Histograms must have the same length")
    
    distance = 0
    if type == 'euclidean': 
        #calculate distance pixelwise
        for i in range(len(hist1)):
            distance += (hist1[i] - hist2[i])**2
        distance = np.sqrt(distance)
    elif type == 'linear':
        for i in range(len(hist1)):
            distance += np.abs(hist1[i] - hist2[i])
        distance /= len(hist1)
    else:
        print("No Distance Calculation Method Specified")
        
#     #CHI SQUARED PSEUDOCODE 
#     distance = 0
#     for i in range(len(hist1)):
#         distance += (hist1[i] - hist2[i]) ** 2
#         distance /= np.max(hist1[i], h2[i])
    print("Distance = ", distance)
    
    return distance

[GO TO TOP](#top)

<a id="color"></a>
-----------------------------
## Color

In [28]:
"""Conversion functions between RGB and other color systems.
This modules provides two functions for each color system ABC:
  rgb_to_abc(r, g, b) --> a, b, c
  abc_to_rgb(a, b, c) --> r, g, b
All inputs and outputs are triples of floats in the range [0.0...1.0]
(with the exception of I and Q, which covers a slightly larger range).
Inputs outside the valid range may cause exceptions or invalid outputs.
Supported color systems:
RGB: Red, Green, Blue components
YIQ: Luminance, Chrominance (used by composite video signals)
HLS: Hue, Luminance, Saturation
HSV: Hue, Saturation, Value
"""

# References:
# http://en.wikipedia.org/wiki/YIQ
# http://en.wikipedia.org/wiki/HLS_color_space
# http://en.wikipedia.org/wiki/HSV_color_space

__all__ = ["rgb_to_yiq","yiq_to_rgb","rgb_to_hls","hls_to_rgb",
           "rgb_to_hsv","hsv_to_rgb"]

# Some floating point constants

ONE_THIRD = 1.0/3.0
ONE_SIXTH = 1.0/6.0
TWO_THIRD = 2.0/3.0

# YIQ: used by composite video signals (linear combinations of RGB)
# Y: perceived gray level (0.0 == black, 1.0 == white)
# I, Q: color components
#
# There are a great many versions of the constants used in these formulae.
# The ones in this library uses constants from the FCC version of NTSC.

def rgb_to_yiq(r, g, b):
    y = 0.30*r + 0.59*g + 0.11*b
    i = 0.74*(r-y) - 0.27*(b-y)
    q = 0.48*(r-y) + 0.41*(b-y)
    return (y, i, q)

def yiq_to_rgb(y, i, q):
    # r = y + (0.27*q + 0.41*i) / (0.74*0.41 + 0.27*0.48)
    # b = y + (0.74*q - 0.48*i) / (0.74*0.41 + 0.27*0.48)
    # g = y - (0.30*(r-y) + 0.11*(b-y)) / 0.59

    r = y + 0.9468822170900693*i + 0.6235565819861433*q
    g = y - 0.27478764629897834*i - 0.6356910791873801*q
    b = y - 1.1085450346420322*i + 1.7090069284064666*q

    if r < 0.0:
        r = 0.0
    if g < 0.0:
        g = 0.0
    if b < 0.0:
        b = 0.0
    if r > 1.0:
        r = 1.0
    if g > 1.0:
        g = 1.0
    if b > 1.0:
        b = 1.0
    return (r, g, b)

# HLS: Hue, Luminance, Saturation
# H: position in the spectrum
# L: color lightness
# S: color saturation

def rgb_to_hls(r, g, b):
    maxc = max(r, g, b)
    minc = min(r, g, b)
    sumc = (maxc+minc)
    rangec = (maxc-minc)
    l = sumc/2.0
    if minc == maxc:
        return 0.0, l, 0.0
    if l <= 0.5:
        s = rangec / sumc
    else:
        s = rangec / (2.0-sumc)
    rc = (maxc-r) / rangec
    gc = (maxc-g) / rangec
    bc = (maxc-b) / rangec
    if r == maxc:
        h = bc-gc
    elif g == maxc:
        h = 2.0+rc-bc
    else:
        h = 4.0+gc-rc
    h = (h/6.0) % 1.0
    return h, l, s

def hls_to_rgb(h, l, s):
    if s == 0.0:
        return l, l, l
    if l <= 0.5:
        m2 = l * (1.0+s)
    else:
        m2 = l+s-(l*s)
    m1 = 2.0*l - m2
    return (_v(m1, m2, h+ONE_THIRD), _v(m1, m2, h), _v(m1, m2, h-ONE_THIRD))

def _v(m1, m2, hue):
    hue = hue % 1.0
    if hue < ONE_SIXTH:
        return m1 + (m2-m1)*hue*6.0
    if hue < 0.5:
        return m2
    if hue < TWO_THIRD:
        return m1 + (m2-m1)*(TWO_THIRD-hue)*6.0
    return m1


# HSV: Hue, Saturation, Value
# H: position in the spectrum
# S: color saturation ("purity")
# V: color brightness

def rgb_to_hsv(r, g, b):
    maxc = max(r, g, b)
    minc = min(r, g, b)
    v = maxc
    if minc == maxc:
        return 0.0, 0.0, v
    s = (maxc-minc) / maxc
    rc = (maxc-r) / (maxc-minc)
    gc = (maxc-g) / (maxc-minc)
    bc = (maxc-b) / (maxc-minc)
    if r == maxc:
        h = bc-gc
    elif g == maxc:
        h = 2.0+rc-bc
    else:
        h = 4.0+gc-rc
    h = (h/6.0) % 1.0
    return h, s, v

def hsv_to_rgb(h, s, v):
    if s == 0.0:
        return v, v, v
    i = int(h*6.0) # XXX assume int() truncates!
    f = (h*6.0) - i
    p = v*(1.0 - s)
    q = v*(1.0 - s*f)
    t = v*(1.0 - s*(1.0-f))
    i = i%6
    if i == 0:
        return v, t, p
    if i == 1:
        return q, v, p
    if i == 2:
        return p, v, t
    if i == 3:
        return p, q, v
    if i == 4:
        return t, p, v
    if i == 5:
        return v, p, q
    # Cannot get here

In [29]:
def split_display_rgb(image, title, display=False, save=False):
    ch_r, ch_g, ch_b = image.split()
    
    if display==True:
        fig = plt.figure()
        plt.subplot(1,3,1); plt.imshow(ch_r, cmap=plt.cm.Reds); plt.axis('off')
        plt.subplot(1,3,2); plt.imshow(ch_g, cmap=plt.cm.Greens); plt.axis('off')
        plt.subplot(1,3,3); plt.imshow(ch_b, cmap=plt.cm.Blues); plt.axis('off')      

        if save == True:
            _ = fig.savefig(title + '.png', dpi = 300, transparent=True)
            
        fig.tight_layout()
        _ = plt.show()
        
    return ch_r, ch_g, ch_b

In [30]:
def grayscale_average_method(image, title, display=False, save=False):
    r, g, b = image.split()
    
    arr_r = np.array(r)
    arr_g = np.array(g)
    arr_b = np.array(b)
    
    
    r_mod = (1/3) * arr_r
    g_mod = (1/3) * arr_g 
    b_mod = (1/3) * arr_b
    
    gray = r_mod + g_mod + b_mod
    
    gray_PIL = Image.fromarray(gray.astype(np.uint8))
    
    if display==True:
        display_image(gray_PIL)
    if save==True:
        gray_PIL.save(title + "_avg.jpg")
    
    return gray_PIL  

In [31]:
def grayscale_luminosity_method(image, title, display=False, save=False):  
    r, g, b = image.split()
    
    arr_r = np.array(r)
    arr_g = np.array(g)
    arr_b = np.array(b)
    
    r_mod = 0.21 * arr_r
    g_mod = 0.72 * arr_g 
    b_mod = 0.07 * arr_b
    
#     r_mod = 0.299 * arr_r
#     g_mod = 0.587 * arr_g 
#     b_mod = 0.114 * arr_b
    
    gray = r_mod + g_mod + b_mod
    
    gray_PIL = Image.fromarray(gray.astype(np.uint8))
    
    if display==True:
        display_image(gray_PIL)
    if save==True:
        gray_PIL.save(title + "_lum.jpg")
    
    return gray_PIL  

In [32]:
def grayscale_lum_gamma_method(image, title=None, gamma=2, display=False, save=False):
    r, g, b = image.split()
    
    arr_r = np.array(r)
    arr_g = np.array(g)
    arr_b = np.array(b)
    
    r_mod = 0.2126 * (arr_r**gamma)
    g_mod = 0.7152 * (arr_g**gamma)
    b_mod = 0.0722 * (arr_b**gamma)
    
    gray = (r_mod + g_mod + b_mod)**(1/gamma)
    
    gray_PIL = Image.fromarray(gray.astype(np.uint8))    
    if display==True:
        display_image(gray_PIL)
    if save==True:
        gray_PIL.save(title + "_lum_gamma.jpg")
    
    return gray_PIL 

[GO TO TOP](#top)

<a id="noise-addition"></a>
-----------------------------
## Noise Addition

In [33]:
def addNoise_gaussian(pil_image=None, mode='', alpha=.5, m=0, v=0.1):
    if pil_image is not None:
        width, height = pil_image.size
        base = np.array(pil_image) # Convert the PIL image to a numpy array
    else:
         # If no image provided, generate a 256x256 base image to use
        width = 256
        height = 256
        base = np.zeros((height, width)) # Create a blank grayscale image
    
    # Generate Gaussian noise
    mean = m
    variance = v
    sigma = np.sqrt(variance)
        
    if mode=='grayscale':
        gaussian = np.random.normal(mean, sigma, (height, width))
        # Scale the Gaussian noise to the range [0, 255]
        gaussian = np.interp(gaussian, (gaussian.min(), gaussian.max()), (0, 255)).astype(np.uint8)
        # Add the Gaussian noise to the image
        combined = (base*(1-alpha) + gaussian*alpha) #.astype(np.uint8)
                
    if mode=='RGB':
        #split by color channel
        red = np.random.normal(mean, sigma, (height, width))
        green = np.random.normal(mean, sigma, (height, width))
        blue = np.random.normal(mean, sigma, (height, width))
        # Stack the noise arrays to form an RGB image
        noise_stack = np.stack([red, green, blue], axis=-1)
        # Scale the noise to the range [0, 255]
        noise = np.interp(noise_stack, (noise_stack.min(), noise_stack.max()), (0, 255)).astype(np.uint8)
        combined = (alpha * noise + (1 - alpha) * base).astype(np.uint8)

    # Convert the numpy array to a PIL image
    outimage = Image.fromarray(combined)

    # Display the image
    #outimage.show()
    
    return outimage

In [34]:
#  Add salt
# and pepper noise to an image
#  @param inimg The input image.
#  @param q The probability. 0 < q < 1 .
#  For each pixel in the image, generate a random number, say r.
#       If (r < q), change the pixel's intensity to zero.
#       If (r > 1-q), change the pixel's intensity to L
#  The higher the q, the worse the noise
#  return Image corrupted by salt and pepper noise.

def addNoise_sap(pil_image=None, q=.05, w=256, h=256):
    # Set seed for random number generator
    random.seed(0)

    if pil_image is not None:
        width, height = pil_image.size
        channel_count = len(pil_image.getbands())
    else:
        # Set image size
        width = w
        height = h
        pil_image = Image.fromarray(np.full((height, width), 128, dtype=np.uint8)) # Create a blank grayscale image
        channel_count = 1
        
    # Create a new output image
    outimage = Image.new(pil_image.mode, pil_image.size)

    # Add SAP noise to each pixel
    for i in range(width):
        for j in range(height):
            for k in range(channel_count):
                r = random.random()
                pixel = pil_image.getpixel((i, j))

                if r < q:
                    outimage.putpixel((i, j), (0,) * channel_count)
                elif r > 1 - q:
                    outimage.putpixel((i, j), (255,) * channel_count)
                else:
                    outimage.putpixel((i, j), pixel)

    # Display the image
    #outimage.show()
    
    return outimage

In [35]:
def addNoise_salt_and_pepper(pil_image=None, mode='', s_vs_p=0.5, amount=0.004):
    if pil_image is not None:
        width, height = pil_image.size
        base = np.array(pil_image) # Convert the PIL image to a numpy array
    else:
        # Set image size
        width = 256
        height = 256
        base = np.full((height, width), 128, dtype=np.uint8) # Create a blank grayscale image
        
    # Generate salt and pepper noise
    noise = np.copy(base)
    num_salt = np.ceil(amount * base.size * s_vs_p)
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in base.shape]
    noise[coords] = 255

    num_pepper = np.ceil(amount* base.size * (1. - s_vs_p))
    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in base.shape]
    noise[coords] = 0
        
    if mode == 'RGB':
        noise_stack = np.stack([noise, noise, noise], axis=-1)
        noise = noise_stack.astype(np.uint8)

    # Convert the numpy array to a PIL image
    outimage = Image.fromarray(noise)

    # Display the image
    #outimage.show()

    return outimage

In [36]:
def addNoise_rayleigh(pil_image=None, alpha = .5, scale=1, m=0):
    if pil_image is not None:
        width, height = pil_image.size
        base = np.array(pil_image) # Convert the PIL image to a numpy array
    else:
        # Set image size
        width = 256
        height = 256
        base = np.full((height, width), 128, dtype=np.uint8) # Create a blank grayscale image
    mean = m
    
    # Convert the numpy array to a PIL image
    #temp = Image.fromarray(base)
    # Display the image
    #temp.show()
    
    
    noise = np.random.rayleigh(scale, size=(width, height))
    noise -= mean
    #noise *= np.sqrt(var / np.var(noise))
    combined = np.uint8(np.clip((alpha * noise + (1 - alpha) * base), 0, 255))
    

    # Convert the numpy array to a PIL image
    outimage = Image.fromarray(combined)
    
    return outimage

In [37]:
def addNoise_uniform(pil_image=None, alpha = .5, scale=1, m=0):
    if pil_image is not None:
        width, height = pil_image.size
        base = np.array(pil_image) # Convert the PIL image to a numpy array
    else:
        # Set image size
        width = 256
        height = 256
        base = np.full((height, width), 128, dtype=np.uint8) # Create a blank grayscale image
    mean = m
    
    # Convert the numpy array to a PIL image
    #temp = Image.fromarray(base)
    # Display the image
    #temp.show()
    
    
    noise = np.random.uniform(scale, size=(width, height))
    noise -= mean
    #noise *= np.sqrt(var / np.var(noise))
    combined = np.uint8(np.clip((alpha * noise + (1 - alpha) * base), 0, 255))
    

    # Convert the numpy array to a PIL image
    outimage = Image.fromarray(combined)
   
    return outimage

[GO TO TOP](#top)

<a id="filtering-denoising"></a>
-----------------------------
## Filtering (Denoising)

In [3]:
def generate_gaussian_kernel(kernel_size=None, sigma=1, display=False):
    
    if kernel_size is not None:
        # Ensure kernel size is odd
        if kernel_size % 2 == 0:
            print("Warning, kernel size has been incremented. Please ensure kernel dimensions are odd.")
            kernel_size += 1
    else: 
        # Auto Calculate the kernel size
        kernel_size = int(6 * sigma + 1) | 1
    
    # Create an empty kernel
    kernel = np.zeros((kernel_size, kernel_size))

    # Calculate the center of the kernel
    center = kernel_size // 2

    # Calculate the constant factor for the Gaussian function
    factor = 1 / (2 * np.pi * sigma ** 2)

    # Populate the kernel with the Gaussian function values
    for i in range(kernel_size):
        for j in range(kernel_size):
            x = i - center
            y = j - center
            kernel[i, j] = factor * np.exp(-(x ** 2 + y ** 2) / (2 * sigma ** 2))

    # Normalize the kernel
    kernel /= np.sum(kernel)
    
    if display:
        print(f"kernel size ({kernel_size}x{kernel_size}), sigma={sigma}:\n", kernel)

    return kernel

In [65]:
def filter_gaussian(image, sigma = 3, alpha = .5):
    # Convert to numpy array
    img = np.array(image)

    # Apply Gaussian filter
    img_blur = cv2.GaussianBlur(img, (0, 0), sigma)
    
    combined = ((1-alpha)*img) + (alpha * img_blur)

    # Convert back to PIL image
    filtered_img = Image.fromarray(combined.astype(np.uint8))
 
    return filtered_img

In [ ]:
def filter_gaussian2(image, sigma=3):
    # Convert the input image to a grayscale NumPy array
    gray = pil_to_numpy(image)

    # Compute the size of the Gaussian kernel
    ksize = int(6 * sigma + 1) | 1

    # Compute the border size
    border = ksize // 2

    # Create the Gaussian kernel
    x, y = np.meshgrid(np.arange(ksize) - border, np.arange(ksize) - border)
    kernel = np.exp(-(x**2 + y**2) / (2 * sigma**2))
    kernel /= np.sum(kernel)
    
    # Pad the image with border pixels
    padded = np.pad(gray, border, mode='edge')

    # Apply the Gaussian filter to the padded image
    filtered = np.zeros_like(gray)
    for i in range(gray.shape[0]):
        for j in range(gray.shape[1]):
            patch = padded[i:i+ksize, j:j+ksize]
            filtered[i, j] = np.sum(patch * kernel)

    # Convert the filtered image back to a PIL Image
    filtered = numpy_to_pil(filtered)

    return filtered, kernel

In [ ]:
def filter_gaussian3(image, sigma):
    """
    Applies a Gaussian filter to the input image with the specified sigma value.
    Args:
        image (PIL.Image): The input image.
        sigma (float): The standard deviation of the Gaussian kernel.
    Returns:
        The filtered image as a NumPy array.
    """
    # Convert the input image to a grayscale NumPy array
    gray = pil_to_numpy(image)

    # Create a Gaussian kernel with the specified sigma value
    size = int(2 * np.ceil(3 * sigma) + 1)
    x, y = np.meshgrid(np.linspace(-1, 1, size), np.linspace(-1, 1, size))
    kernel = np.exp(-(x**2 + y**2) / (2 * sigma**2))
    kernel /= np.sum(kernel)

    # Pad the image with reflective padding to avoid boundary effects
    pad_size = int(2 * np.ceil(3 * sigma))
    padded = np.pad(gray, pad_size, mode='reflect')

    # Apply the Gaussian filter to the padded image
    filtered = np.zeros_like(gray)
    for i in range(gray.shape[0]):
        for j in range(gray.shape[1]):
            patch = padded[i:i+size, j:j+size]
            filtered[i, j] = np.sum(patch * kernel)

    # Convert the filtered image back to a PIL Image
    filtered = numpy_to_pil(filtered)
    
    return filtered

In [ ]:
def filter_gaussian4(image, sigma):
    """
    Applies a Gaussian filter to the input image with the specified sigma value.
    Args:
        image (PIL.Image): The input image.
        sigma (float): The standard deviation of the Gaussian kernel.
    Returns:
        The filtered image as a NumPy array.
    """
    # Convert the input image to a grayscale NumPy array
    gray = pil_to_numpy(image)

    # Compute the size of the kernel
    size = int(2 * np.ceil(3 * sigma) + 1)

    # Create the Gaussian kernel
    x, y = np.meshgrid(np.linspace(-1, 1, size), np.linspace(-1, 1, size))
    kernel = np.exp(-(x**2 + y**2) / (2 * sigma**2))
    kernel /= np.sum(kernel)

    # Pad the image with symmetric padding to avoid boundary effects
    pad_size = size // 2
    padded = np.pad(gray, pad_size, mode='symmetric')

    # Apply the Gaussian filter to the padded image
    filtered = np.zeros_like(gray)
    for i in range(gray.shape[0]):
        for j in range(gray.shape[1]):
            patch = padded[i:i+size, j:j+size]
            filtered[i, j] = np.sum(patch * kernel)

    # Convert the filtered image back to a PIL Image
    filtered = numpy_to_pil(filtered)
    return filtered

In [ ]:
def filter_gaussian5(image, sigma):
    """
    Applies a Gaussian filter to the input image with the specified sigma value.
    Args:
        image (PIL.Image): The input image.
        sigma (float): The standard deviation of the Gaussian kernel.
    Returns:
        The filtered image as a NumPy array.
    """
    # Convert the input image to a grayscale NumPy array
    gray = pil_to_numpy(image)

    # Create a Gaussian kernel with the specified sigma value
    size = int(2 * np.ceil(3 * sigma) + 1)
    x, y = np.meshgrid(np.linspace(-1, 1, size), np.linspace(-1, 1, size))
    kernel = np.exp(-(x**2 + y**2) / (2 * sigma**2))
    kernel /= np.sum(kernel)
    
    # Pad the image with zeros to avoid border effects
    padded = np.pad(gray, size, mode='constant')

    # Apply the Gaussian filter to the padded image
    filtered = np.zeros_like(gray)
    for i in range(gray.shape[0]):
        for j in range(gray.shape[1]):
            patch = padded[i:i+size, j:j+size]
            filtered[i, j] = np.sum(patch * kernel)

    # Convert the filtered image back to a PIL Image
    filtered = numpy_to_pil(filtered)
    return filtered

In [39]:
def filter_bilateral(image, title='', sigma_spatial=10, fileName=''):
    # Convert image to numpy array
    img = np.array(image)
    
    # Create meshgrid of pixel coordinates
    x, y = np.meshgrid(np.arange(img.shape[1]), np.arange(img.shape[0]))
    
    # Initialize filtered image
    filtered_img = np.zeros_like(img)
    
    # Iterate over each pixel in the image
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            # Calculate spatial weights using Euclidean distance
            spatial_weights = np.exp(-((x - x[i,j]) ** 2 + (y - y[i,j]) ** 2) / (2 * sigma_spatial ** 2))
            
            # Calculate range weights using intensity difference
            range_weights = np.exp(-(img - img[i,j]) ** 2 / (2 * sigma_spatial ** 2))
            
            # Calculate combined weights
            weights = range_weights * spatial_weights
            
            # Normalize weights
            weight_sum = np.sum(weights)
            if weight_sum == 0:
                # If the sum of weights is zero, set weights to a uniform distribution
                weights = np.ones_like(weights) / weights.size
            else:
                # Otherwise, normalize weights as usual
                weights /= weight_sum
            
            # Apply weights to image
            filtered_img[i,j] = np.sum(weights * img)
            
    # Convert filtered image back to PIL image
    filtered_img = Image.fromarray(filtered_img)
    
    # Show original and filtered images using matplotlib
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
    plt.axis('off')
    axs[0].imshow(img, cmap='gray')
    axs[0].set_title(title + ' Noisy')
    axs[1].imshow(filtered_img, cmap='gray')
    axs[1].set_title(title + ' Filtered (Sigma Spatial = {})'.format(sigma_spatial))
        
    fig.tight_layout()
    _ = fig.savefig(fileName + '.png', dpi = 300, transparent=True)
    plt.show()
    
    return filtered_img

In [40]:
def filter_bilateral_fast(img, title='', sigma_spatial=10, fileName=''):
    # Convert image to numpy array
    img = np.array(img)

    # Create meshgrid of pixel coordinates
    x, y = np.meshgrid(np.arange(img.shape[1]), np.arange(img.shape[0]))

    # Initialize filtered image
    filtered_img = np.zeros_like(img)

    # Calculate spatial weights using Euclidean distance
    spatial_weights = np.exp(-((x - x[..., np.newaxis, np.newaxis]) ** 2 + (y - y[..., np.newaxis, np.newaxis]) ** 2) / (2 * sigma_spatial ** 2))

    # Iterate over each pixel in the image
    for c in range(img.shape[2]):
        # Calculate range weights using intensity difference
        range_weights = np.exp(-(img - img[..., c][..., np.newaxis, np.newaxis]) ** 2 / (2 * sigma_spatial ** 2))

        # Calculate combined weights
        weights = range_weights * spatial_weights

        # Normalize weights
        weight_sum = np.sum(weights, axis=(0, 1))
        weights /= weight_sum

        # Apply weights to image
        filtered_img[..., c] = np.sum(weights * img, axis=(0, 1))

    # Convert filtered image back to PIL image
    filtered_img = Image.fromarray(filtered_img)

    # Show original and filtered images using matplotlib
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
    plt.axis('off')
    axs[0].imshow(img, cmap='gray')
    axs[0].set_title(title + ' Noisy')
    axs[1].imshow(filtered_img, cmap='gray')
    axs[1].set_title(title + ' Filtered (Sigma Spatial = {})'.format(sigma_spatial))

    fig.tight_layout()
    _ = fig.savefig(fileName + '.png', dpi = 300, transparent=True)
    plt.show()

    return filtered_img

In [41]:
def filter_geometric(img, title='', kernel_size=3, fileName=''):
    # Convert image to numpy array
    img = np.array(img)
    
    # Initialize filtered image
    filtered_img = np.zeros_like(img)
    
    # Iterate over each pixel in the image
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            # Get kernel centered at current pixel
            kernel = img[max(0, i-kernel_size//2):min(img.shape[0], i+kernel_size//2+1),
                          max(0, j-kernel_size//2):min(img.shape[1], j+kernel_size//2+1)]
            # Calculate geometric mean of kernel
            kernel_log = np.log(kernel + 0.001)  # add a small constant to avoid zero or negative values
            filtered_img[i,j] = np.exp(np.mean(kernel_log))
    
    # Convert filtered image back to PIL image
    filtered_img = Image.fromarray(filtered_img)
    
    # Show original and filtered images using matplotlib
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
    plt.axis('off')
    axs[0].imshow(img, cmap='gray')
    axs[0].set_title(title + ' Noisy')
    axs[1].imshow(filtered_img, cmap='gray')
    axs[1].set_title(title + ' Filtered (Kernel Size = {})'.format(kernel_size))
        
    fig.tight_layout()
    #_ = fig.savefig(fileName + '.png', dpi=300, transparent=True)
    plt.show()
    
    return filtered_img

In [42]:
def filter_geo_alpha_trimmed(img, d=10, kernel_size=3):
    # Convert image to numpy array
    img = np.array(img)
    
    # Initialize filtered image
    filtered_img = np.zeros_like(img)
    
    # Iterate over each pixel in the image
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            # Get kernel centered at current pixel
            kernel = img[max(0, i-kernel_size//2):min(img.shape[0], i+kernel_size//2+1),
                          max(0, j-kernel_size//2):min(img.shape[1], j+kernel_size//2+1)]
            # Sort pixel values in kernel
            sorted_kernel = np.sort(kernel.flatten())
            # Exclude d/2 largest and d/2 smallest values
            trimmed_kernel = sorted_kernel[d//2:-d//2]
            # Calculate geometric mean of trimmed kernel
            kernel_log = np.log(trimmed_kernel + 0.001)  # add a small constant to avoid zero or negative values
            filtered_img[i,j] = np.exp(np.mean(kernel_log))

    # Convert filtered image back to PIL image
    filtered_img = Image.fromarray(filtered_img)
    
    return filtered_img

In [43]:
def filter_median(data, kernel_size=3, boundary_mode='reflect'):
    # Check kernel size is odd
    if kernel_size % 2 == 0:
        raise ValueError("Kernel size must be odd")
    
    # Apply median filter with custom parameters
    filtered_img = median_filter(data, size=kernel_size, mode=boundary_mode)
    
    return filtered_img

In [44]:
def filter_mmm(image, kernel_size=3, filter_type='median'):
     # Convert image to numpy array
    img = np.array(image)
    
    # Pad the image with zeros to handle edges
    pad_width = kernel_size // 2
    padded_image = np.pad(img, pad_width, mode='constant', constant_values=0)
    
    # Define the filter function based on the filter type
    if filter_type == 'median':
        filter_func = lambda x: np.median(x)
    elif filter_type == 'max':
        filter_func = lambda x: np.max(x)
    elif filter_type == 'min':
        filter_func = lambda x: np.min(x)
    else:
        raise ValueError('Invalid filter type. Allowed values are "median", "max", and "min".')
    
    # Apply the filter to each pixel in the image
    filtered_image = np.zeros_like(img)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            neighborhood = padded_image[i:i+kernel_size, j:j+kernel_size]
            filtered_image[i, j] = filter_func(neighborhood)
    
    # Crop the image to its original size
    cropped_image = filtered_image[pad_width:-pad_width, pad_width:-pad_width]
    
    # Convert filtered image back to PIL image
    filtered_img = Image.fromarray(cropped_image)
    
    return filtered_img

In [45]:
def filter_weighted_moving_avg(image, kernel):
     # Convert image to numpy array
    img = np.array(image)
    
    # Get kernel size
    kernel_size = len(kernel)

    # Pad image to handle borders
    pad_width = kernel_size // 2
    #padded_image = np.pad(image, ((pad_width, pad_width), (pad_width, pad_width)), mode='edge')
    padded_image = np.pad(img, pad_width, mode='constant', constant_values=0)
    
    # Initialize output image
    filtered_image = np.zeros_like(img)

    # Apply filter to each pixel
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            patch = padded_image[i:i+kernel_size, j:j+kernel_size]
            filtered_image[i, j] = np.sum(patch * kernel)
            
    # Crop the image to its original size
    cropped_image = filtered_image[pad_width:-pad_width, pad_width:-pad_width]
            
    # Convert filtered image back to PIL image
    filtered_img = Image.fromarray(cropped_image)
    
    return filtered_image

In [46]:
def filter_alpha_trimmed(img, d=10, kernel_size=3, alpha=0.2):
    # Convert image to numpy array
    img = np.array(img)
    
    # Initialize filtered image
    filtered_img = np.zeros_like(img)
    
    # Iterate over each pixel in the image
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            # Get kernel centered at current pixel
            kernel = img[max(0, i-kernel_size//2):min(img.shape[0], i+kernel_size//2+1),
                          max(0, j-kernel_size//2):min(img.shape[1], j+kernel_size//2+1)]
            # Sort pixel values in kernel
            sorted_kernel = np.sort(kernel.flatten())
            # Exclude alpha fraction of the largest and smallest values
            trimmed_kernel = sorted_kernel[int(alpha*len(sorted_kernel)):int((1-alpha)*len(sorted_kernel))]
            # Calculate mean of trimmed kernel
            filtered_img[i,j] = np.mean(trimmed_kernel)

    # Convert filtered image back to PIL image
    filtered_img = Image.fromarray(filtered_img.astype(np.uint8))
    
    return filtered_img

In [47]:
def filter_nonlinear(image, alpha=0.2, kernel_size=3):
    # Convert image to numpy array
    img = np.array(image)
    
    # Initialize filtered image
    filtered_img = np.zeros_like(img)
    
    # Iterate over each pixel in the image
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            # Get kernel centered at current pixel
            kernel = img[max(0, i-kernel_size//2):min(img.shape[0], i+kernel_size//2+1),
                          max(0, j-kernel_size//2):min(img.shape[1], j+kernel_size//2+1)]
            # Sort pixel values in kernel
            sorted_kernel = np.sort(kernel.flatten())
            # Exclude alpha fraction of the largest and smallest values
            trimmed_kernel = sorted_kernel[int(alpha*len(sorted_kernel)):-int(alpha*len(sorted_kernel))]
            # Calculate geometric mean of trimmed kernel
            kernel_log = np.log(trimmed_kernel + 0.001)  # add a small constant to avoid zero or negative values
            filtered_img[i,j] = np.exp(np.mean(kernel_log))

    # Convert filtered image back to PIL image
    filtered_img = Image.fromarray(filtered_img)
    
    return filtered_img

[GO TO TOP](#top)

<a id="filtering-sharpening"></a>
-----------------------------
## Filtering (Sharpening)

In [48]:
'''
image can be PIL or numpy
kernel is a weighted array:
    exe   0  2  0
          2  5  2
          0  2  0
alpha is the "strength" of the high pass filter's application to the og image
'''

def sharpen(image, kernel, alpha):
    # Convert image to numpy array
    img = np.array(image)
    
    # Normalize kernel weights to sum to 1
    #kernel = kernel / (np.sum(kernel)+1e-8) #add small const to prevent divide by 0
    
    # Pad the image with zeros to handle edges
    pad_width = len(kernel) // 2
    padded_image = np.pad(img, pad_width, mode='constant', constant_values=0)
    
    # Apply the kernel to each pixel in the image
    filtered_image = np.zeros_like(img)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            neighborhood = padded_image[i:i+len(kernel), j:j+len(kernel)]
            filtered_image[i, j] = np.sum(neighborhood * kernel)
    
    # Subtract the filtered image from the original to get the high-pass filtered image
    highpass_image = img - filtered_image
    
    # Add the high-pass filtered image back to the original with a weight of 0.5 to create the sharpened image
    sharpened_image = ((1-alpha)*img) + (alpha * highpass_image)
    
    # Convert filtered image back to PIL image
    sharpened_img = Image.fromarray(sharpened_image)
    
    return sharpened_img

-----------------------------

[GO TO TOP](#top)

<a id="thresholding--object-detection"></a>
## Thresholding | Object Detection

In [49]:
def single_threshold(image, T):
    # Convert PIL image to numpy array if necessary
    if isinstance(image, Image.Image) and image.mode != 'L':
        image = pil_to_numpy(image.convert('L'))
    else:
        image = pil_to_numpy(image)
        
    # Apply thresholding using boolean indexing
    thresholded_image = np.zeros_like(image)
    thresholded_image[image > T] = 255

    # Convert to PIL type
    thresholded_image = Image.fromarray(thresholded_image.astype('uint8'))

    return thresholded_image

In [50]:
def multi_threshold(image, *T):
    # Convert PIL image to numpy array if necessary
    if isinstance(image, Image.Image) and image.mode != 'L':
        image = pil_to_numpy(image.convert('L'))
    else:
        image = pil_to_numpy(image)

    # Initialize output image
    thresholded_image = np.zeros_like(image)

    # Calculate threshold values as average of closest T values
    thresholds = [0] * (len(T) + 1)
    for i in range(len(T) + 1):
        if i == 0:
            thresholds[i] = T[i] / 2
        elif i == len(T):
            thresholds[i] = (T[i-1] + 255) / 2
        else:
            thresholds[i] = (T[i-1] + T[i]) / 2

    # Create threshold categories
    for i in range(len(thresholds) - 1):
        threshold_min = thresholds[i]
        threshold_max = thresholds[i+1]
        threshold_value = (threshold_min + threshold_max) // 2
        thresholded_image[(image >= threshold_min) & (image < threshold_max)] = threshold_value
    # Convert to PIL type
    thresholded_image = Image.fromarray(thresholded_image.astype('uint8'))

    return thresholded_image

In [51]:
def threshold_moving_average(image, neighborhood=21, weight=.5):
    # Convert image to grayscale
    img = np.array(image)

    # Define the neighborhood for thresholding
    win_size = neighborhood

    # Define the weight parameter for threshold calculation
    k = weight

    # Apply moving average thresholding
    th = np.zeros_like(img)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            # Calculate the local threshold using the moving average
            mean = np.mean(img[max(0, i-win_size//2):min(img.shape[0], i+win_size//2+1), max(0, j-win_size//2):min(img.shape[1], j+win_size//2+1)])
            th_val = mean * (1 + k)

            # Apply thresholding
            if img[i,j] > th_val:
                th[i,j] = 255
            else:
                th[i,j] = 0

    # Convert back to PIL Image
    th = Image.fromarray(th)

    return th

In [52]:
def threshold_gaussian(image, neighborhood=21, sigma=3, k=0.1):
    # Define the image and kernel
    img = np.array(image)
    kernel = np.zeros((neighborhood, neighborhood))
    center = neighborhood // 2

    # Create a Gaussian kernel
    for i in range(neighborhood):
        for j in range(neighborhood):
            kernel[i,j] = np.exp(-((i-center)**2 + (j-center)**2)/(2*sigma**2))

    # Normalize the kernel
    kernel /= np.sum(kernel)

    # Add padding to the image
    pad_size = neighborhood // 2
    img_padded = np.pad(img, ((pad_size, pad_size), (pad_size, pad_size)), mode='reflect')

    # Apply Gaussian filtering to the padded image
    img_filtered = gaussian_filter(img_padded, sigma=sigma)

    # Apply adaptive thresholding
    thresholded = np.zeros_like(img)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            # Calculate the threshold
            threshold = k * np.mean(kernel * img_filtered[i:i+neighborhood, j:j+neighborhood])

            # Apply thresholding
            if img[i,j] > threshold:
                thresholded[i,j] = 255
            else:
                thresholded[i,j] = 0

    thresholded = Image.fromarray(thresholded)
    return thresholded

In [3]:
def professor_provided_thresholding(image):
    if image is None:
        print("file could not be read, check with os.path.")
    img = image
    img = cv2.medianBlur(img, 5)
    th1 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
    th2 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    th3 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)

    titles = ['Original Image', 'Adaptive Mean Thresholding', 'Adaptive Gaussian Thresholding', 'Global Thresholding']

    images = [img, th1, th2, th3]

    for i in range(4):
        plt.subplot(2,2,i+1)
        plt.imshow(images[i], cmap='gray')
        plt.title(titles[i])
        plt.xticks([])
        plt.yticks([])
    plt.imshow()

In [5]:
def otsu_best_threshold(image):
    if isinstance(image, np.ndarray):
        # If image is a NumPy array, do nothing
        im = image  
    elif isinstance(image, Image.Image) and image.mode == 'L':
        # If image is a PIL Image object, convert it to a NumPy array
        im = pil_to_numpy(image)
    else:
        raise ValueError("Image format not supported. Please provide a NumPy array or a PIL Image object.")
        
    threshold_range = np.arange(np.max(im)+1)
    criterias = []
    
    for th in threshold_range:
        # create the thresholded image
        thresholded_im = np.zeros(im.shape)
        thresholded_im[im >= th] = 1

        # compute weights
        nb_pixels = im.size
        nb_pixels1 = np.count_nonzero(thresholded_im)
        weight1 = nb_pixels1 / nb_pixels
        weight0 = 1 - weight1

        # if one the classes is empty, eg all pixels are below or above the threshold, that threshold will not be considered
        # in the search for the best threshold
        if weight1 == 0 or weight0 == 0:
            continue

        # find all pixels belonging to each class
        val_pixels1 = im[thresholded_im == 1]
        val_pixels0 = im[thresholded_im == 0]

        # compute variance of these classes
        var0 = np.var(val_pixels0) if len(val_pixels0) > 0 else 0
        var1 = np.var(val_pixels1) if len(val_pixels1) > 0 else 0
        criterias.append(weight0 * var0 + weight1 * var1)
        
    # best threshold is the one minimizing the Otsu criteria    
    best_thresh = np.argmin(criterias)
    
    return best_thresh

In [9]:
def otsu_best_threshold(image):
    if isinstance(image, np.ndarray):
        # If image is a NumPy array, do nothing
        im = image  
    elif isinstance(image, Image.Image) and image.mode == 'L':
        # If image is a PIL Image object, convert it to a NumPy array
        im = pil_to_numpy(image)
    else:
        raise ValueError("Image format not supported. Please provide a NumPy array or a PIL Image object.")
        
    # Compute histogram of the image using cv2
    hist = cv2.calcHist([im], [0], None, [256], [0, 256]).flatten()
    nb_pixels = im.size

    # Compute cumulative sum of the histogram
    cum_hist = np.cumsum(hist)
    cum_hist_inv = np.cumsum(hist[::-1])[::-1]

    # Compute cumulative mean, avoiding division by zero
    cum_mean = np.cumsum(hist * np.arange(256)) / np.maximum(cum_hist, 1)
    cum_mean_inv = (np.cumsum((hist * np.arange(256))[::-1])[::-1]) / np.maximum(cum_hist_inv, 1)

    # Compute inter-class variance
    variance = cum_hist[:-1] * cum_hist_inv[1:] * (cum_mean[:-1] - cum_mean_inv[1:])**2

    # Find threshold that maximizes inter-class variance
    best_thresh = np.argmax(variance)
    
    return best_thresh


[GO TO TOP](#top)

<a id="niblack-bernsen-sauvola"></a>
### Niblack, Bernsen, Sauvola

In [54]:
def threshold_niblack(image, neighborhood = 21, weight = -0.2):
    #define our image
    img = np.array(image.convert('L'))
    # Define the neighborhood for thresholding
    win_size = neighborhood
    # Define the weight parameter for threshold calculation
    k = weight

    # Apply Niblack thresholding
    th = np.zeros_like(img)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            # Calculate the local threshold using mean and standard deviation
            mean, std_dev = cv2.meanStdDev(img[max(0, i-win_size//2):min(img.shape[0], i+win_size//2+1),
                                               max(0, j-win_size//2):min(img.shape[1], j+win_size//2+1)])
            th_val = mean + k * std_dev

            # Apply thresholding
            if img[i,j] > th_val:
                th[i,j] = 255
            else:
                th[i,j] = 0
    th = Image.fromarray(th)
    return th

In [55]:
def threshold_sauvola(image, neighborhood = 21, weight = -0.2):
    #define our image
    img = np.array(image)
    # Define the neighborhood for thresholding
    win_size = neighborhood
    # Define the weight parameter for threshold calculation
    k = weight

    # Define the R parameter for threshold calculation
    R = 128

    # Apply Sauvola thresholding
    th = np.zeros_like(img)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            # Calculate the local threshold using mean and standard deviation
            mean, std_dev = cv2.meanStdDev(img[max(0, i-win_size//2):min(img.shape[0], i+win_size//2+1),
                                               max(0, j-win_size//2):min(img.shape[1], j+win_size//2+1)])
            th_val = mean * (1 + k * ((std_dev / R) - 1))

            # Apply thresholding
            if img[i,j] > th_val:
                th[i,j] = 255
            else:
                th[i,j] = 0

    th = Image.fromarray(th)
    return th

In [56]:
def threshold_bernsen(image, neighborhood=21, contrast_threshold=15):
    # Convert the image to grayscale
    img = np.array(image)

    # Apply Bernsen thresholding
    th = np.zeros_like(img)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            # Calculate the local contrast using the min and max pixel values in the neighborhood
            window_min = np.min(img[max(0, i-neighborhood//2):min(img.shape[0], i+neighborhood//2+1),
                                     max(0, j-neighborhood//2):min(img.shape[1], j+neighborhood//2+1)])
            window_max = np.max(img[max(0, i-neighborhood//2):min(img.shape[0], i+neighborhood//2+1),
                                     max(0, j-neighborhood//2):min(img.shape[1], j+neighborhood//2+1)])
            contrast = window_max - window_min

            # Apply thresholding based on local contrast
            if contrast < contrast_threshold:
                threshold = 255 // 2
            else:
                threshold = (window_min + window_max) // 2

            # Apply thresholding
            if img[i,j] > threshold:
                th[i,j] = 255
            else:
                th[i,j] = 0

    th = Image.fromarray(th)
    return th

[GO TO TOP](#top)

<a id="convolution--edge-detection"></a>
-----------------------------
## Convolution & Edge Detection

In [71]:
'''
GENERAL CONVOLUTION - Grayscale Only
Takes:
    - img: PIL or numpy array
    - kernel (numpy.ndarray): A nxn kernel as a NumPy array.
Returns:
    - PIL Image
'''
def convolve_image(image, kernel, save=False, title=''):
    img = np.array(image)
    height, width = img.shape
    k_height, k_width = kernel.shape

    #Zero-Padding
    pad_height = k_height // 2
    pad_width = k_width // 2
    padded = np.pad(img, ((pad_height, pad_height), (pad_width, pad_width)), mode='constant')
    # Initialize the output image with zeros
    output = np.zeros((height, width))

    for i in range(height):
        for j in range(width):
            # Get the corresponding region in the padded image
            region = padded[i:i+k_height, j:j+k_width]
            # Perform element-wise multiplication between the region and the kernel
            result = region * kernel
            # Calculate the sum of the resulting matrix
            output[i, j] = np.sum(result)
            
    #Trim zero-padded pixels
#     output = output[pad_height:height+pad_height, pad_width:width+pad_width]

    image_convoluted = Image.fromarray(output)
    
    if save == True:
        _ = fig.savefig(title + '.png', dpi = 300, transparent=True)
        
    return image_convoluted

In [68]:
def convolve_image_gradient(image, x_kernel, y_kernel):
    img = np.array(image)
    height, width = img.shape
    k_height, k_width = x_kernel.shape
    assert x_kernel.shape == y_kernel.shape

    #Zero-Padding
    pad_height = k_height // 2
    pad_width = k_width // 2
    padded = np.pad(img, ((pad_height, pad_height), (pad_width, pad_width)), mode='constant')
    
    # Initialize the output images with zeros
    x_output = np.zeros((height, width))
    y_output = np.zeros((height, width))

    for i in range(height):
        for j in range(width):
            # Get the corresponding region in the padded image
            region = padded[i:i+k_height, j:j+k_width]
            # Perform element-wise multiplication between the region and the x_kernel
            x_result = region * x_kernel
            # Perform element-wise multiplication between the region and the y_kernel
            y_result = region * y_kernel
            # Calculate the sum of the resulting matrices
            x_output[i, j] = np.sum(x_result)
            y_output[i, j] = np.sum(y_result)
            
    #Trim zero-padded pixels
    x_output = x_output[pad_height:height+pad_height, pad_width:width+pad_width]
    y_output = y_output[pad_height:height+pad_height, pad_width:width+pad_width]

    x_difference = Image.fromarray(x_output)
    y_difference = Image.fromarray(y_output)

    return x_difference, y_difference

In [71]:
#sobel display
def process_image_with_sobel(im, threshold_values, display=False, save=False):
    # Convert input image to grayscale
    im_gray = im.convert('L')
    
    # Compute x and y gradients using Sobel operator
    x_diff, y_diff = convolve_image_gradient(im_gray, kernel3x3_sobel_x, kernel3x3_sobel_y)
    if save == True:
        save_image(x_diff, 'sobel_x')
        save_image(y_diff, 'sobel_y')
    
    # Compute gradient magnitude and maximum
    x_diff_np = np.array(x_diff)
    y_diff_np = np.array(y_diff)
    gradient_mag = ((x_diff_np**2) + (y_diff_np**2))**.5
    gradient_max = np.maximum(x_diff_np, y_diff_np)
    if save == True: 
        save_image(gradient_mag, 'sobel_mag')
        save_image(gradient_max, 'sobel_max')
    
    # Compute the gradient direction in degrees
    direction = np.degrees(np.arctan2(y_diff_np, x_diff_np))
    grad_dir_PIL = Image.fromarray(direction.astype(np.uint8))
    if save == True:
        save_image(grad_dir_PIL, 'grad_direction_in_deg')
    
    # Apply thresholding to gradient magnitude and maximum for each threshold value
    for threshold in threshold_values:
        sobel_mag = single_threshold(Image.fromarray(gradient_mag), threshold)
        sobel_max = single_threshold(Image.fromarray(gradient_max), threshold)
        
        # Save thresholded images with descriptive filenames
        sobel_mag_filename = f'sobel_mag_T{threshold}'
        sobel_max_filename = f'sobel_max_T{threshold}'
        if save == True:
            save_image(sobel_mag, sobel_mag_filename)
            save_image(sobel_max, sobel_max_filename)
        
        # Display thresholded images using display_pair function
        if display == True: 
            display_pair(sobel_mag, f'sobel mag T{threshold}', save, 6, 2)
            display_pair(sobel_max, f'sobel max T{threshold}', save, 6, 2)
        
    return sobel_max, sobel_mag

In [63]:
# #sobel display
# def process_image_with_sobel(image, threshold_values, display=False, save=False):
#     np_image = pil_to_numpy(image)
    
#     # Compute x and y gradients using Sobel operator
#     x_diff, y_diff = convolve_image_gradient(np_image, kernel3x3_sobel_x, kernel3x3_sobel_y)
#     if save == True: 
#         save_image(x_diff, 'sobel_x.jpg')
#         save_image(y_diff, 'sobel_y.jpg')
    
#     # Compute gradient magnitude and maximum
#     x_diff_np = pil_to_numpy(x_diff)
#     y_diff_np = pil_to_numpy(y_diff)
#     gradient_mag = ((x_diff_np**2) + (y_diff_np**2))**.5
#     gradient_max = np.maximum(x_diff_np, y_diff_np)
#     if save == True: 
#         save_image(gradient_mag, 'sobel_mag.jpg')
#         save_image(gradient_max, 'sobel_max.jpg')
    
#     # Compute the gradient direction in degrees
#     direction = np.degrees(np.arctan2(y_diff_np, x_diff_np))
    
#     # More PIL conversion
#     gradient_mag_PIL = numpy_to_pil(gradient_mag)
#     gradient_max_PIL = numpy_to_pil(gradient_max)
#     grad_dir_PIL = numpy_to_pil(direction.astype(np.uint8))
#     if save == True:
#         save_image(grad_dir_PIL, 'grad_direction_in_deg.jpg')
    
#     # Apply thresholding to gradient magnitude and maximum for each threshold value
#     for threshold in threshold_values:
#         sobel_mag = single_threshold(gradient_mag_PIL, threshold)
#         sobel_max = single_threshold(gradient_max_PIL, threshold)
        
#         # Save thresholded images with descriptive filenames
#         sobel_mag_filename = f'sobel_mag_T{threshold}.jpg'
#         sobel_max_filename = f'sobel_max_T{threshold}.jpg'
#         if save == True:
#             save_image(sobel_mag, sobel_mag_filename)
#             save_image(sobel_max, sobel_max_filename)
        
#         # Display thresholded images using display_pair function
#         if display == True:
#             display_pair(sobel_mag, f'sobel mag T{threshold}', 6, 2, save)
#             display_pair(sobel_max, f'sobel max T{threshold}', 6, 2, save)
        
#     return sobel_max, sobel_mag

In [67]:
#sobel display
def process_batch_with_sobel(images, threshold_values, set='', display=False):
    
    # Convert input image to grayscale
    sobelmax = []
    
    for image in images:
        # Compute x and y gradients using Sobel operator
        x_diff, y_diff = convolve_image_gradient(image, kernel3x3_sobel_x, kernel3x3_sobel_y)
        save_image(x_diff, 'sobel_x')
        save_image(y_diff, 'sobel_y')

        # Compute gradient magnitude and maximum
        x_diff_np = np.array(x_diff)
        y_diff_np = np.array(y_diff)
        gradient_mag = ((x_diff_np**2) + (y_diff_np**2))**.5
        gradient_max = np.maximum(x_diff_np, y_diff_np)
#         save_image(gradient_mag, 'sobel_mag')
#         save_image(gradient_max, 'sobel_max')
        
        # Compute the gradient direction in degrees
        direction = np.degrees(np.arctan2(y_diff_np, x_diff_np))
        grad_dir_PIL = Image.fromarray(direction.astype(np.uint8))
#         save_image(grad_dir_PIL, 'grad_direction_in_deg')

        # Apply thresholding to gradient magnitude and maximum for each threshold value
        for threshold in threshold_values:
            sobel_mag = single_threshold(Image.fromarray(gradient_mag), threshold)
            sobel_max = single_threshold(Image.fromarray(gradient_max), threshold)
            
            sobelmax.append(sobel_max)
            
            # Save thresholded images with descriptive filenames
#             sobel_mag_filename = f'sobel_mag_T{threshold}'
#             sobel_max_filename = f'sobel_max_T{threshold}'
#             save_image(sobel_mag, sobel_mag_filename)
#             save_image(sobel_max, sobel_max_filename)

            # Display thresholded images using display_pair function
#             display_pair(sobel_mag, f'sobel mag T{threshold}', thresh_save_status, 6, 2)
            if display == True:
                display_pair(sobel_max, f'sobel max T{threshold}', False, 6, 2)
        
    return sobelmax

In [74]:
def process_image_with_gaussian(image, sigma = 1, alpha=1, threshold_values=[40,80]):
    # convert to grayscale
#     image_gray = image.convert('L')

    # Apply Gaussian blurring
    blurred = filter_gaussian(image_gray, sigma, alpha)
    save_image(blurred, f'blurred_S{sigma}.jpg')
    display_image(blurred)

    # Apply Sobel convolution
    x_diff, y_diff = convolve_image_gradient(blurred, kernel3x3_sobel_x, kernel3x3_sobel_y)
    save_image(x_diff, 'x_diff_gauss.jpg')
    save_image(y_diff, 'y_diff_gauss.jpg')

    # Compute the gradient magnitude
    x_diff_np = np.array(x_diff)
    y_diff_np = np.array(y_diff)
    gradient_mag = ((x_diff_np ** 2) + (y_diff_np ** 2)) ** 0.5
    gradient_mag_PIL = Image.fromarray(gradient_mag)
    save_image(gradient_mag_PIL, 'grad_magnitude.jpg')

    

    # Compute the gradient direction in degrees
    grad_dir = np.degrees(np.arctan2(y_diff_np, x_diff_np))
    grad_dir_PIL = Image.fromarray(grad_dir.astype(np.uint8))
    save_image(grad_dir_PIL, 'grad_direction_in_deg.jpg')

    # Loop over threshold values and apply thresholding
    for threshold in threshold_values:
        thresholded = single_threshold(gradient_mag_PIL, threshold)
        save_image(thresholded, f'grad_mag_thresh{threshold}.jpg')
        display_image(thresholded)

In [76]:
def process_image_with_canny(image, sigma = 0.75, alpha=1, threshold_low_values=[40,80], threshold_high_values=[120, 200]):
    # convert to grayscale
    image_gray = image.convert('L')

    # Apply Gaussian blurring
    blurred = filter_gaussian(image_gray, sigma, alpha)
    save_image(blurred, f'blurred_S{sigma}.jpg')
    display_image(blurred)
    blurred = np.array(blurred)

    # Apply Sobel convolution
    x_diff, y_diff = convolve_image_gradient(blurred, kernel3x3_sobel_x, kernel3x3_sobel_y)
    save_image(x_diff, 'x_diff_gauss.jpg')
    save_image(y_diff, 'y_diff_gauss.jpg')

    # Compute the gradient magnitude
    x_diff_np = np.array(x_diff)
    y_diff_np = np.array(y_diff)
    gradient_mag = ((x_diff_np ** 2) + (y_diff_np ** 2)) ** 0.5
    gradient_mag_PIL = Image.fromarray(gradient_mag)
    save_image(gradient_mag_PIL, 'grad_magnitude.jpg')

    # Compute the gradient direction in degrees
    grad_dir = np.degrees(np.arctan2(y_diff_np, x_diff_np))
    grad_dir_PIL = Image.fromarray(grad_dir.astype(np.uint8))
    save_image(grad_dir_PIL, 'grad_direction_in_deg.jpg')
    
    # Loop over threshold values and apply thresholding
    for threshold_low in threshold_low_values:
        for threshold_high in threshold_high_values:
            canny = canny = cv2.Canny(blurred, threshold_low, threshold_high)
            save_image(canny, f'canny_thresh{threshold_low}_{threshold_high}.jpg')
            display_image(canny)

In [9]:
def process_robinson_direction(image, direction=None, threshold=30, return_feats=False, display=False, save=False):
    
    if direction is None:
        raise TypeError("direction should not = None \nUse: n, nw, w, sw, se, e, ne")
        
    elif direction == 'n':
    #-----------------NORTH----------------------
        robinson_image = convolve_image(image, kernel3x3_robinson_N)
        robinson_edge = single_threshold(robinson_image, threshold)
    
    elif direction == 'nw':
    #-----------------NORTHWEST----------------------
        robinson_image = convolve_image(image, kernel3x3_robinson_NW)
        robinson_edge = single_threshold(robinson_image, threshold)
    
    elif direction == 'w':
    #-----------------WEST----------------------
        robinson_image = convolve_image(image, kernel3x3_robinson_W)
        robinson_edge = single_threshold(robinson_image, threshold)

    elif direction == 'sw':
    #-----------------SOUTHWEST----------------------
        robinson_image = convolve_image(image, kernel3x3_robinson_SW)
        robinson_edge = single_threshold(robinson_image, threshold)

    elif direction == 's':
    #-----------------SOUTH----------------------
        robinson_image = convolve_image(image, kernel3x3_robinson_S)
        robinson_edge = single_threshold(robinson_image, threshold)

    elif direction == 'se':
    #-----------------SOUTHEAST----------------------
        robinson_image = convolve_image(image, kernel3x3_robinson_SE)
        robinson_edge = single_threshold(robinson_image, threshold)

    elif direction == 'e':
    #-----------------EAST----------------------
        robinson_image = convolve_image(image, kernel3x3_robinson_E)
        robinson_edge = single_threshold(robinson_image, threshold)

    elif direction == 'ne':
    #-----------------NORTHEAST----------------------
        robinson_image = convolve_image(image, kernel3x3_robinson_NE)
        robinson_edge = single_threshold(robinson_image, threshold)

    
    if display == True:
        display_pair(robinson_image, f'Robinson Image {direction}', False, 6,2)
        display_pair(robinson_edge, f'Robinson Edge {direction} T{threshold}', False, 6, 2)
 
    
    if save == True:
        save_image(robinson_image, f"{direction}_img_T{threshold}.jpg")
        save_image(robinson_edge, f"{direction}_edge_T{threshold}.jpg")
        
    if return_feats:
        robinson_feats = pil_to_numpy(robinson_edge).ravel()
        return robinson_feats
 
    return robinson_image, robinson_edge

In [27]:
def process_image_with_robinson(image, threshold=30, display=False, save=False):
    #-----------------NORTH----------------------
    north = convolve_image(image, kernel3x3_robinson_N)
    n_edge = single_threshold(north, threshold)
    
    #-----------------NORTHWEST----------------------
    northwest = convolve_image(image, kernel3x3_robinson_NW)
    nw_edge = single_threshold(northwest, threshold)

    #-----------------WEST----------------------
    west = convolve_image(image, kernel3x3_robinson_W)
    w_edge = single_threshold(west, threshold)

    #-----------------SOUTHWEST----------------------
    southwest = convolve_image(image, kernel3x3_robinson_SW)
    sw_edge = single_threshold(southwest, threshold)

    #-----------------SOUTH----------------------
    south = convolve_image(image, kernel3x3_robinson_S)
    s_edge = single_threshold(south, threshold)

    #-----------------SOUTHEAST----------------------
    southeast = convolve_image(image, kernel3x3_robinson_SE)
    se_edge = single_threshold(southeast, threshold)

    #-----------------EAST----------------------
    east = convolve_image(image, kernel3x3_robinson_E)
    e_edge = single_threshold(east, threshold)

    #-----------------NORTHEAST----------------------
    northeast = convolve_image(image, kernel3x3_robinson_NE)
    ne_edge = single_threshold(northeast, threshold)

    
    if display == True:
        display_pair(north, 'North', False, 6,2)
        display_pair(northwest, 'Northwest')
        display_pair(west, 'West', False, 6, 2)
        display_pair(southwest, 'Southwest')
        display_pair(south, 'South', False, 6, 2)
        display_pair(southeast, 'Southeast')
        display_pair(east, 'East', False, 6, 2)
        display_pair(northeast, 'Northeast') 
    
    if save == True:
        save_image(n_edge, f"northT{threshold}.jpg")
        save_image(nw_edge, f"northwestT{threshold}.jpg")
        save_image(w_edge, f"westT{threshold}.jpg")
        save_image(sw_edge, f"southwestT{threshold}.jpg")
        save_image(s_edge, f"southT{threshold}.jpg")
        save_image(se_edge, f"southeastT{threshold}.jpg")
        save_image(e_edge, f"eastT{threshold}.jpg")
        save_image(ne_edge, f"northeastT{threshold}.jpg")
        
    return n_edge, nw_edge, w_edge, sw_edge, s_edge, se_edge, e_edge, ne_edge

In [3]:
def hog_automated(image, o=8, ppc=(16, 16), cpb=(1, 1), v=True, display=False):
    img_np = pil_to_numpy(image)
    hog_feature, hog_image = hog(img_np, orientations=o, pixels_per_cell=ppc, 
                                 cells_per_block=cpb, visualize=v, channel_axis=None)
    hog_image1_rescaled = exposure.rescale_intensity(hog_image, in_range=(0,10))
   
    #plot
    if display:
        fig, (axes1, axes2) = pylab.subplots(1, 2, figsize=(15, 10), sharex=True, sharey=True)
        axes1.axis('off'), axes1.imshow(img_np, cmap=pylab.cm.gray),
        #axes1.set_title('Input image')
        axes2.axis('off'), axes2.imshow(hog_image1_rescaled, cmap=pylab.cm.gray),
        #axes2.set_title('Histogram of Oriented Gradients')
        axes2.figure.savefig('hog_image_rescaled.png', bbox_inches='tight', pad_inches=0, transparent=True)
        pylab.show()

    return hog_feature, hog_image

In [1]:
def hog_batch(images, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1)):
    hog_np_images = []
    hog_features = []
    
    for image in images:
        img_np = np.array(image)
        fd, hog_image = hog(img_np, orientations, pixels_per_cell, cells_per_block)
        hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0,10))
        hog_np_images.append(hog_image_rescaled)
        hog_features.append(hog_features)
        
    return hog_features, hog_np_images

In [107]:
def display_hog(images, set_id=None, display=False, save=False):
    # retrieve plot colors from main file
    %store -r plt_color
    %store -r label_color
    %store -r title_color

    hog_features = []
    hog_images = []
    
    #     images = pil_to_numpy(images)
    images_badcoder = []
    if isinstance(images, (list, tuple, np.ndarray)):
        images_badcoder = images
        img_count = len(images)
    else:
        images_badcoder.append(images)
        img_count = 1   
    
    i=1
    for image in images_badcoder:
        # Convert the PIL image to a NumPy array
        np_image = np.array(image)

        # Compute the HoG representation of the image
        hog_feature, hog_image = hog(np_image, orientations=8, pixels_per_cell=(16, 16),
                            cells_per_block=(1, 1), visualize=True, channel_axis=None)
    
        hog_features.append(hog_feature)
        hog_images.append(hog_image)
        
        # Display the original image and its HoG representation side by side
        if display == True:
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5), sharex=True, sharey=True)
            ax1.imshow(image, cmap=plt.cm.gray)
            ax1.axis('off')
            ax1.set_title(f'{set_id}{i}', color = title_color)
            ax2.imshow(hog_image, cmap=plt.cm.gray)
            ax2.axis('off')
            ax2.set_title('Histogram of Oriented Gradients', color = title_color)
            if save == True:
                _ = fig.savefig(f'{set_id}{i}_hog.png', dpi = 300, transparent=True)
            plt.show()
        i+=1
    
    hog_images = numpy_to_pil(hog_images)
    return hog_features, hog_images

[GO TO TOP](#top)

<a id="morphology"></a>
-----------------------------
## Morphology

In [4]:
# SHAPE_DICT = {
#                 "RECT": cv2.MORPH_RECT,
#                 "CROSS": cv2.MORPH_CROSS,
#                 "ELLIPSE": cv2.MORPH_ELLIPSE,
#                 "HITMISS": cv2.MORPH_HITMISS,
#             }

In [5]:
def dilation(image, disk_sizes=[5, 10], shape="rect", display = False, save = False):
    # Convert PIL image to NumPy array
    img = np.array(image)

    # Check if shape is valid
    if shape.upper() not in SHAPE_DICT:
        raise ValueError("Invalid shape specified")

    # Get the cv2.MORPH_* value for the specified shape
    shape_val = SHAPE_DICT[shape.upper()]

    dilated_images = []
    for disk in disk_sizes:
        # Warn user about inaccurate kernels
        if disk % 2 == 0 and shape == 'ellipse':
            print(f"WARNING: detected even disk size {disk} for ellipse kernel.",
                  "\n\tThis structure will not generate symmetricly.")
                
        # Perform dilation with disk
        structure = cv2.getStructuringElement(shape_val, (disk, disk))
        BW = cv2.dilate(img, structure)
    
        # Display the dilated image
        if display == True:
            print("\nStructure:\n",structure)
            display_image(BW, '', False, 3)

        # Save the dilated image
        if save == True:
            save_image(BW, f'dilation_{shape}{disk}.jpg')

        # Convert NumPy array to PIL image
        dilated_images.append(Image.fromarray(BW))

    return tuple(dilated_images)

In [6]:
def dilate_batch(images, disk_sizes=[5, 10], shape="rect", display = False, save = False):
    # Convert PIL image to NumPy array

    pil_dilated = []
    
    # Check if shape is valid
    if shape.upper() not in SHAPE_DICT:
        raise ValueError("Invalid shape specified")

    # Get the cv2.MORPH_* value for the specified shape
    shape_val = SHAPE_DICT[shape.upper()]

    for image in images:
        img = np.array(image)
        for disk in disk_sizes:
            # Warn user about inaccurate kernels
            if disk % 2 == 0 and shape == 'ellipse':
                print(f"WARNING: detected even disk size {disk} for ellipse kernel.",
                  "\n\tThis structure will not generate symmetricly.")
            
            # Perform dilation with disk
            structure = cv2.getStructuringElement(shape_val, (disk, disk))
            BW = cv2.dilate(img, structure)

            # Display the dilated image
            if display == True:
                print("\nStructure:\n",structure)
                display_image(BW, '', False, 3)

            # Save the dilated image
            if save == True:
                save_image(BW, f'dilation_{shape}{disk}.jpg')

            # Convert NumPy array to PIL image
            BW = Image.fromarray(BW)
        pil_dilated.append(BW)

    return pil_dilated

In [7]:
def erosion(image, disk_sizes=[5, 10], shape="rect", display = False, save = False):
    # Convert PIL image to NumPy array
    img = np.array(image)

    # Check if shape is valid
    if shape.upper() not in SHAPE_DICT:
        raise ValueError("Invalid shape specified")

    # Get the cv2.MORPH_* value for the specified shape
    shape_val = SHAPE_DICT[shape.upper()]

    eroded_images = []
    for disk in disk_sizes:
        # Warn user about inaccurate kernels
        if disk % 2 == 0 and shape == 'ellipse':
            print(f"WARNING: detected even disk size {disk} for ellipse kernel.",
                  "\n\tThis structure will not generate symmetricly.")
                
        # Perform dilation with disk
        structure = cv2.getStructuringElement(shape_val, (disk, disk))
        BW = cv2.erode(img, structure)
    
        # Display the dilated image
        if display == True:
            print("\nStructure:\n",structure)
            display_image(BW, '', False, 3)

        # Save the dilated image
        if save == True:
            save_image(BW, f'erosion_{shape}{disk}.jpg')

        # Convert NumPy array to PIL image
        eroded_images.append(Image.fromarray(BW))

    return tuple(eroded_images)

In [8]:
def erode_batch(images, disk_sizes=[5, 10], shape="rect", display = False, save = False):
    # Convert PIL image to NumPy array

    pil_eroded = []
    
    # Check if shape is valid
    if shape.upper() not in SHAPE_DICT:
        raise ValueError("Invalid shape specified")

    # Get the cv2.MORPH_* value for the specified shape
    shape_val = SHAPE_DICT[shape.upper()]

    for image in images:
        img = np.array(image)
        for disk in disk_sizes:
            # Warn user about inaccurate kernels
            if disk % 2 == 0 and shape == 'ellipse':
                print(f"WARNING: detected even disk size {disk} for ellipse kernel.",
                  "\n\tThis structure will not generate symmetricly.")
                
            # Perform dilation with disk
            structure = cv2.getStructuringElement(shape_val, (disk, disk))
            BW = cv2.erode(img, structure)

            # Display the dilated image
            if display == True:
                print("\nStructure:\n",structure)
                display_image(BW, '', False, 3)

            # Save the dilated image
            if save == True:
                save_image(BW, f'erosion_{shape}{disk}.jpg')

            # Convert NumPy array to PIL image
            BW = Image.fromarray(BW)
        pil_eroded.append(BW)

    return pil_eroded

In [9]:
#erode then dilate
def opening(image, disk_sizes=[5, 10], shape="rect", show_struct=False, display=False, save=False):
    # Convert PIL image to NumPy array
    img = np.array(image)

    # Check if shape is valid
    if shape.upper() not in SHAPE_DICT:
        raise ValueError("Invalid shape specified")

    # Get the cv2.MORPH_* value for the specified shape
    shape_val = SHAPE_DICT[shape.upper()]
    
    opened_images = []
    for disk in disk_sizes:
        # Warn user about inaccurate kernels
        if disk % 2 == 0 and shape == 'ellipse':
            print(f"WARNING: detected even disk size {disk} for ellipse kernel.",
                  "\n\tThis structure will not generate symmetricly.")
            
        # Get Structural Element
        structure = cv2.getStructuringElement(shape_val, (disk, disk))
    
        # Perform Closing (Dilation then Erosion)
        eroded = cv2.erode(img, structure)
        opened = cv2.dilate(eroded, structure)
        
        # Show the structural element
        if show_struct: 
            print("\nStructure:\n",structure)
            
        # Display the dilated image
        if display == True:
            display_image(eroded, 'Eroded before Opening', False, 3)
            display_image(opened, 'Opened', False, 3)

        # Save the dilated image
        if save == True:
            save_image(eroded, f'eroded_{shape}{disk}.jpg')
            save_image(opened, f'opened_{shape}{disk}.jpg')

        # Convert NumPy array to PIL image
        opened_images.append(Image.fromarray(opened))

    return tuple(opened_images)

In [10]:
#dilate then erode
def closing(image, disk_sizes=[5, 10], shape="rect", show_struct=False, display=False, save=False):
    # Convert PIL image to NumPy array
    img = np.array(image)

    # Check if shape is valid
    if shape.upper() not in SHAPE_DICT:
        raise ValueError("Invalid shape specified")

    # Get the cv2.MORPH_* value for the specified shape
    shape_val = SHAPE_DICT[shape.upper()]
    
    closed_images = []
    for disk in disk_sizes:
        # Warn user about inaccurate kernels
        if disk % 2 == 0 and shape == 'ellipse':
            print(f"WARNING: detected even disk size {disk} for ellipse kernel.",
                  "\n\tThis structure will not generate symmetricly.")
        # Get Structural Element
        structure = cv2.getStructuringElement(shape_val, (disk, disk))
        
        # Perform Closing (Dilation then Erosion)
        dilated = cv2.dilate(img, structure)
        closed = cv2.erode(dilated, structure)
        
        # Show the structural element
        if show_struct: 
            print("\nDisk:\n",disk,"\nStructure:\n",structure)
            
        # Display the dilated image
        if display == True:
            display_image(dilated, 'Dilated Before Closing', False, 3)
            display_image(closed, 'Closed', False, 3)

        # Save the dilated image
        if save == True:
            save_image(dilated, f'dilated_{shape}{disk}.jpg')
            save_image(closed, f'closed_{shape}{disk}.jpg')

        # Convert NumPy array to PIL image
        closed_images.append(Image.fromarray(closed))

    return tuple(closed_images)

[GO TO TOP](#top)

<a id="midterm-general"></a>
-----------------------------
## Midterm General

In [80]:
# BATCH PROCESS
#images should be np images
def matching_CDF(images, set='', display=False, save=False):
    np_matched = []
    
    if detect_type(images) not in ('numpy_set', 'pil_set'):
        raise TypeError("matching_CDF function has not recieved a set of images to compare.")
    
    i=1
    total_dist = 0
    for image in images:
        #histogram matching
        match = hist_matching(image, cdf(image), cdf(clean16combo))
        
        #add to matched array
        np_matched.append(match)
        
        #display
        match = Image.fromarray(match)
        total_dist += image_distance(clean16combo, match, display_hist=False)   
        if display == True:
            display_pair(match, f"{set}{i} + Match", save, m=6, n=2)
            display_hist(match, f"{set}{i}_histmatch", save=save)
        if save == True:
            save_image(match, f"{set}{i}_histmatch.jpg")
        
        i+=1
        
    avg_dist = total_dist / i
    print("AVERAGE DISTANCE =", avg_dist)
    return np_matched

In [81]:
def apply_mask(image, slope, start_height, fill_value, mask_color=0):
    
    width, height = image.size
    new_image = Image.new(image.mode, image.size)
    draw = ImageDraw.Draw(new_image)
    
    x1 = 0
    y1 = int(start_height)
    x2 = width
    y2 = int(start_height + slope * (width - 1))
    
    mask_image = Image.new('L', image.size, fill_value)
    mask_draw = ImageDraw.Draw(mask_image)
    mask_draw.polygon([(x1, y1), (x2, y2), (x2, height), (x1, height)], fill=255 - fill_value)
    masked_image = Image.composite(image, Image.new('L', image.size, mask_color), mask_image)
    merged_image = Image.alpha_composite(new_image.convert('RGBA'), masked_image.convert('RGBA'))
    
    return merged_image.convert('L')

In [1]:
# Example usage
def crop_to_panel(images, mask_color=0, set_id=None):
    images = numpy_to_pil(images)
    cropped_set = []
    
    fill1 = 255
    fill2 = 0
    fill3 = 0 
    
    if detect_type(images) == 'pil_single':
        cropped = apply_mask(images, 0.17, 135, fill1, mask_color)
        cropped = apply_mask(cropped, 0.15, 10, fill2, mask_color)
        cropped = apply_mask(cropped, -2, 180, fill3, mask_color)
        return cropped
    else:
        counter=0
        for image in images:
            cropped = apply_mask(image, 0.17, 135, fill1, mask_color)
            cropped = apply_mask(cropped, 0.15, 10, fill2, mask_color)
            cropped = apply_mask(cropped, -2, 180, fill3, mask_color)
            cropped_set.append(cropped)
#             #save the 15th image
#             if counter == 15:
#                 save_image(cropped, f"{set_id}{counter}_cropped.jpg")            
#             counter += 1
        return cropped_set

[GO TO TOP](#top)

<a id="midterm-procedures"></a>
-----------------------------
## Midterm Procedures

In [36]:
def solar_system(image_set, benchmark = None, set_id=None, 
                 part1_params=None, part2_params=None, part3_params=None, 
                 part4_params=None, part5_params=None, part6_params=None, part7_params=None,
                 abort_after=None, 
                 display_originals = True, display_steps = True, display_finals = True, 
                 fileName=None, save=True):

    # ======================================= SETUP =======================================    
    
    #Determine if image_set is a single image or a list/tuple/np.ndarray
    if isinstance(image_set, (list, tuple, np.ndarray)):
        set_size = len(image_set)
    else:
        set_size = 1
        
    #Display Input Set
    if display_originals:
        titles = [f"{set_id}{i+1} Og." for i in range(set_size)]
        plot_nxn(image_set, titles=titles, fileName=f'{set_id}_Original', save=save)

    # ================================ BEGIN STEPS =========================================
    #---------------PART 1------------------------------------------------------------------
    #-input project database images---------------------------------------------------------
    #---------------------------------------------------------------------------------------
    pil_step1 = image_set
    #pil_step1 = selection_procedure(...)
       
    #Abort 1 Procedure
    if abort_after == 1:
        #image_set_final = pil_step1
        print("\nProcessing aborted after Part 1 \n\tReturned image(s) are unmodified.\n")
        return pil_step1
    
    #---------------PART 2------------------------------------------------------------------
    #-apply image restoration and enhancement methods to improve quality--------------------
    #---------------------------------------------------------------------------------------
    pil_step2 = pil_step1
    pil_step2 = enhancement_procedure(pil_step2, f'{set_id}', params=part2_params)
    
    #Display step
    if display_steps:
        plot_nxn(pil_step2, fileName = f'STEP 2 - {set_id} Enhancement', save=True)
    
    #Abort 2 Procedure
    if abort_after == 2:
        print("\nProcessing aborted after Part 2 \n\tReturned image(s) are enhanced.\n")
        return pil_step2
    
    #---------------PART 3------------------------------------------------------------------
    #-retreive binary segmented images via segmentation methods-----------------------------
    #---------------------------------------------------------------------------------------
    pil_step3 = pil_step2
    pil_step3 = segmentation_procedure(pil_step3, set_id=set_id, params=part3_params)
    
    #Display step
    if display_steps:
        plot_nxn(pil_step3, fileName = f'STEP 3 - {set_id} Segmentation', save=True)
    
    #Abort 3 Procedure
    if abort_after == 3:
        print("\nProcessing aborted after Part 3 \n\tReturned image(s) are binary segmented.\n")
        return pil_step3
    
    #---------------PART 4------------------------------------------------------------------
    #-obtain isolated object in via multiplication of (enhanced * segmented binary)---------
    #---------------------------------------------------------------------------------------
    pil_step4 = pil_step3
    pil_step4 = multiplicative_procedure(pil_step2, pil_step4, set_id=set_id, params=part4_params)
    
    #Display step
    if display_steps:
        plot_nxn(pil_step4, fileName = f'STEP 4 - {set_id} Multiplicative', save=True)
    
    #Abort 4 Procedure
    if abort_after == 4:
        print("\nProcessing aborted after Part 4 \n\tReturned image(s) are gray-level segmented (object isolated).\n")
        return pil_step4
    
    #---------------PART 5------------------------------------------------------------------
    #-generate feature information (Histogram or HoG) via segmented gray-level images-------
    #---------------------------------------------------------------------------------------
    pil_step5 = pil_step4
    pil_step5, features, benchmark_features = feature_procedure(pil_step5, benchmark, set_id=set_id, params=None)
    
    #Display step
    if display_steps:
        print("image features[0]: ", features[0])
        print("benchmark features: ", benchmark_features)

    #Abort 5 Procedure
    if abort_after == 5:
        print("\nProcessing aborted after Part 5.",
              " \n\tHistogram data has been generated.",
              "\n\tReturned \n\t\t- image(s) are gray-level segmented (object isolated)",
             "\n\t\t- feature data (list) \n\t\t- benchmark features (single)\n")
        return pil_step5, features, benchmark_features
    
    #---------------PART 6------------------------------------------------------------------
    #-similarity data has been calculated using a benchmark img & segmented gray images-----
    #---------------------------------------------------------------------------------------       
    pil_step6 = pil_step5
    pil_step6, hist_distances, feat_distances = similarity_procedure(pil_step6, benchmark, features, benchmark_features,
                                                             set_id=set_id, params=None)
    #Display step
    if display_steps:
        print("Image Histogram Distances\n", hist_distances)
        print("Image Feature Distances\n", feat_distances)
    
    #Abort 6 Procedure
    if abort_after == 6:
        print("\nProcessing aborted after Part 6",
              " \n\tSimilarity data has been calculated.",
              "\n\tReturned \n\t\t- image(s) are gray-level segmented (object isolated)",
             "\n\t\t- histogram difference calculations \n\t\t- feature distance calculations\n")
        return pil_step6, hist_distances, feat_distances
    
    #---------------PART 7------------------------------------------------------------------
    #---------------------------------------------------------------------------------------
    #---------------------------------------------------------------------------------------  
    pil_step7 = pil_step6
    #pil_step7 = accuracy_evaluation_procedure(...)
    
    #Abort 7 Procedure
    if abort_after == 7:
        print("\nProcessing aborted after Part 7",
              " \n\tClassification accuracy has been evaluated. Returned image(s) are gray-level segmented (object isolated).\n")
        return pil_step7
    
    
    # ================================== END STEPS ===========================================
    #Display final set images if desired.
    image_set_final = pil_step7
    print("All 7 steps have completed.")
    if display_finals:
        titles = [f"{set_id}{i+1} Final" for i in range(set_size)]
        plot_nxn(image_set_final, titles=titles, fileName=f'{set_id}_Final', save=save)

    return image_set_final

In [4]:
# STEP 2 - Enhancement
def enhancement_procedure(images, set_id=None, params=None):
    """
    This function performs image enhancement on a set of images using a combination of histogram stretching, gamma adjustment, and equalization.

    Parameters:
    images (PIL.Image.Image or list of PIL.Image.Image or numpy.ndarray or list of numpy.ndarray): An instance of PIL.Image.Image or a list of instances of PIL.Image.Image, or a 2D numpy array or a list of 2D numpy arrays representing the image(s) to enhance.
    set_id (str): An optional string to use as a prefix for display and/or save filenames.
    params (dict): An optional dictionary containing custom parameters to use for the enhancement.

    Returns:
    list (PIL.Image.Image(s)): representing the enhanced images
    """
    
    np_enhanced = []
    print("STEP 2: Performing the following tasks...")
    
    # DEFAULT PARAMETERS 
    print("- setting default parameters: gamma, alpha, hist_clip, equalize_clip, display, save")
    gamma = 0.75
    alpha = 0.5
    hist_clip = 0.005
    equalize_clip = 0.03
    display = False
    save = False

    # UNPACK CUSTOM PARAMETERS
    if params is not None:
        print("- unpacking custom parameters")
        print("ERROR: Custom Parameters not set up for this step. Terminating Process")
        return images

    # CONVERSIONS
    images = pil_to_numpy(images)    # if data is PIL, convert it to NumPy

    i=1
    total_dist = 0
    print("- histogram stretching")
    print("- gamma adjustment")
    print("- equalization")
    print("- image alpha combination")
    for image in images:
        #histogram stretch + clipping
        image_stretch = histogram_stretch_clip(image, hist_clip)
        image_gamma = image_stretch.point(lambda x: 255*(x/255)**gamma)  
        image_gamma = np.array(image_gamma).astype(np.uint8)
        
#         #stretch again (NEW ADDITION)
#         image_stretch2 = histogram_stretch_clip(image_gamma, hist_clip)
#         image_gamma = image_stretch2
        
        #equalization
        image_equalized = exposure.equalize_hist(image)*255
        # image_equalized = exposure.equalize_adapthist(image, clip_limit=equalize_clip)*255

        #combine both parts
        part1 = pil_to_numpy(image_gamma)
        part2 = pil_to_numpy(image_equalized)
        image_combo = (alpha)*part1 + (1-alpha)*part2
        
        #append to array
        np_enhanced.append(image_combo)

        #display
        image_combo = Image.fromarray(image_combo)
        # total_dist += image_distance(clean16combo, image_combo, display_hist=False)   
        if display == True:
            display_pair(image_combo, f"{set_id}{i} + Combo", save, m=6, n=2)
            display_hist(image_combo, f"{set_id}{i}_enhanced", save=save)      
        if save == True:
            save_image(image_combo, f"{set_id}{i}_enhanced.jpg")
        
        i+=1
    
    pil_enhanced = numpy_to_pil(np_enhanced)
    
    return pil_enhanced

In [27]:
# STEP 3 - Segmentation
def segmentation_procedure(images, set_id=None, params=None):
    """
    This function performs image segmentation on a set of images using various methods including single T thresholding and Sobel segmentation.

    Parameters:
    images (numpy.ndarray or list of numpy.ndarray): A 2D numpy array or a list of 2D numpy arrays representing the grayscale image(s) to segment.
    set_id (str): An optional string to use as a prefix for display and/or save filenames, must be either 'clean' or 'dirty'
    params (dict): An optional dictionary containing custom parameters to use for the segmentation.
    
    Returns:
    list (PIL.Image.Image(s)): representing the segmented images
    """
    segmented = []
    otsu_values = []
    print("STEP 3: Performing the following tasks...")
    
    # DEFAULT PARAMETERS
    print("- setting default parameters: segmentation_method, otsu_predefined, invert_region,",
              "\n\t\tmorphology, disk, manual_crop, crop_bg_value, display, save")
    segmentation_method = None
    otsu_predefined = None
    invert_region = False
    morphology = None
    disk = None
    manual_crop = True
    crop_bg_value = 0
    display = False
    save = False
    
    # UNPACK CUSTOM PARAMETERS
    if params is not None:
        print("- unpacking custom parameters")
        segmentation_method = params.get("segmentation_method")
        otsu_predefined = params.get("otsu_predefined")
        invert_region = params.get("invert_region")
        morphology = params.get("morphology")
        disk = params.get("disk")
        manual_crop = params.get("manual_crop")
        crop_bg_value = params.get("crop_bg_value")
        display = params.get("display")
        save = params.get("save")
        
    if set_id == 'clean':
        dilate_disk = [10]
        erode_disk = [6]
        open_disk = [10]
        close_disk = [10]
    elif set_id == 'dirty':
        dilate_disk = [4]
        erode_disk = [6]
        open_disk = [10]
        close_disk = [10]
    elif disk is not None:
        print(f"\t\tWARNING! Because set is neither \'clean\' nor \'dirty\', Disk size used = {disk}.")
    elif morphology is not None:
        raise NameError("ERROR! When DISK = NONE, set_id must be either \'clean\' or \'dirty\'")
        return images;
        
    # CONVERSIONS
    images = numpy_to_pil(images)  

#     # CROPPING
#     if manual_crop:
#         print("- cropping")
#         images = crop_to_panel(images, crop_bg_value)
    
    # ----------- SEGMENTATION ---------
    counter = 1
    
    # SINGLE T HALFWAY
    if segmentation_method == 'single_T_halfway':
        print("- single thresholding at 128")
        for image in images:
            single_thresh = single_threshold(image, [128])
            segmented.append(single_thresh)     
            if save == True:
                save_image(single_thresh, f'{set_id}{counter}_single_T{best_thresh}')
            counter += 1 
    
    # SINGLE T SEGMENTATION 
    elif segmentation_method == 'single_T':
        print("- single thresholding via otsu T")
        for image in images:
            best_thresh = otsu_best_threshold(image)
            otsu_values.append(best_thresh)

            single_thresh = single_threshold(image, [best_thresh])
            segmented.append(single_thresh)     
            if save == True:
                save_image(single_thresh, f'{set_id}{counter}_single_T{best_thresh}')
            counter += 1 
            
    # SOBEL SEGMENTATION       
    elif segmentation_method == 'sobel':
        print("- sobel segmentation + otsu thresholding")
        for image in images:
            best_thresh = otsu_best_threshold(image)
            otsu_values.append(best_thresh)

            sobel_max, sobel_mag = process_image_with_sobel(image, [best_thresh], display=display, save=save)
            segmented.append(sobel_max)     
            if save:
                save_image(sobel_max, f'{set_id}{counter}_sobel_max_T{best_thresh}')
            counter += 1    
    else:
        print("- NOTICE! No segmentation performed.")
        segmented = images
    
    # INVERSION
    if invert_region:
        print("- inversion")
        invert_segmented = []

        for image in segmented:
            temp = pil_to_numpy(image)
            inverted = np.abs(255 - temp)
            inverted = numpy_to_pil(inverted)
            invert_segmented.append(inverted)
            segmented = invert_segmented

        if display:
            plot_nxn(segmented, fileName='invert_region', save=True)  
    
    # MORPHOLOGY    
    if morphology == 'dilate':
        print("- morphological dilaton")
        if disk == None: 
            segmented = dilate_batch(segmented, disk_sizes=dilate_disk, display=display, save=save)
        else: 
            segmented = dilate_batch(segmented, disk_sizes=disk, display=display, save=save)

    elif morphology == 'erode':
        print("- morphological erosion")
        if disk == None:
            segmented = erode_batch(segmented, disk_sizes=erode_disk, display=display, save=save)
        else:
            segmented = erode_batch(segmented, disk_sizes=disk, display=display, save=save)
            
#     elif morphology == 'open':
#            print("- morphological opening")     
#         segmented = 
#     elif morphology == 'close':
#            print("- morphological closing")
#         segmented =

    # CROPPING     
    if manual_crop:
        print("- cropping")
        segmented = crop_to_panel(segmented, crop_bg_value) 
        
    # RETURN PROPERLY SIZED IMAGES
    segmented = resize_images(segmented, height=192, width=192, display_status=False)

    segmented = numpy_to_pil(segmented)
    return segmented

In [26]:
# STEP 4 - Multiplicative Gray-Level Segmentation
#future improvement: images_binary should be a parameter. images_gray should be renamed to images.
def multiplicative_procedure(images_gray, images_binary, set_id=None, params=None):
    """
    Parameters:
    images_gray (numpy.ndarray or list of numpy.ndarray): A 2D numpy array or a list of 2D numpy arrays representing the grayscale image(s) to segment.
    images_binary (numpy.ndarray or list of numpy.ndarray): A 2D numpy array or a list of 2D numpy arrays representing the binary segmented image(s) used to enhance the grayscale image.
    set_id (str): An optional string to use as a prefix for display and/or save filenames, must be either 'clean' or 'dirty'
    params (dict): An optional dictionary containing custom parameters to use for the multiplicative segmentation.

    Returns:
    list (PIL.Image.Image(s)): representing the multiplied images.
    """
    multiplied = []
    print("STEP 4: Performing the following tasks...")
    
    # DEFAULT PARAMETERS
    print("- setting default parameters: display, save")
    display = False
    save = False    
    # UNPACK CUSTOM PARAMETERS
    if params is not None:
        print("- unpacking custom parameters")
        print("ERROR: Custom Parameters not set up for this step. Terminating Process")
        return images_binary
    
    # CONVERSIONS & RESIZING
    images_gray = numpy_to_pil(images_gray)  
    images_binary = numpy_to_pil(images_binary)
    print("- resizing images to 192x192")
    images_gray = resize_images(images_gray, height=192, width=192, display_status=False)
    images_binary = resize_images(images_binary, height=192, width=192, display_status=False)
        
    images_gray = pil_to_numpy(images_gray)  
    images_binary = pil_to_numpy(images_binary)

    # MULTIPLICATION
    counter = 1
    print("- multiplying (grayscale enhanced * binary segmented) images")
    for im_g, im_b in zip(images_gray, images_binary):
        gray_masked = im_g * im_b
        multiplied.append(gray_masked)     
        if save == True:
            save_image(gray_masked, f'{set_id}{counter}_multiplied')
        counter += 1
    
    multiplied = numpy_to_pil(multiplied)
    return multiplied

In [34]:
# STEP 5 - Feature Retrieval (histogram data or HoG)
def feature_procedure(images, benchmark=None, set_id=None, params=None):
    """
    Parameters:
    images (numpy.ndarray or list of numpy.ndarray): A 2D numpy array or a list of 2D numpy arrays representing the image(s) to retrieve features from.
    benchmark (numpy.ndarray or None): An optional 2D numpy array representing the benchmark image used for feature comparison.
    set_id (str): An optional string to use as a prefix for display and/or save filenames, must be either 'clean' or 'dirty'
    params (dict): An optional dictionary containing custom parameters to use for feature retrieval.

    Returns:
    tuple (numpy.ndarray or list of numpy.ndarray, list, numpy.ndarray or None): 
        - images: A 2D numpy array or a list of 2D numpy arrays representing the image(s) passed as input.
        - feature_set: A list of feature data generated from the input image(s) using histogram data or HoG.
        - benchmark_features: A 1D numpy array representing the feature data generated from the benchmark image, if provided. Returns None if benchmark is not provided.

    """
    feature_set = []
    print("STEP 5: Performing the following tasks...")

    # DEFAULT PARAMETERS
    print("- setting default parameters: feature_type, display, save, use_hog")
    feature_type = 'hog'
    display = False
    save = False
    # UNPACK PARAMETERS
    if params is not None:
        print("- unpacking custom parameters")
        print("ERROR: Custom Parameters not set up for this step. Terminating Process")
        return images
        
    # CONVERSIONS
    images = numpy_to_pil(images)  

    # RETRIEVE FEATURE DATA
    if feature_type == 'hist':
        print("- Histogram feature data generation")
        for image in images:
            hist = image.histogram()
            if display == True:
                display_hist(image)
            feature_set.append(hist)
        if benchmark is not None:
            benchmark_features = benchmark.histogram()
    elif feature_type == 'hog':
        print("- HoG feature data generation")
        hog_features, hog_images = display_hog(images, set_id=set_id, display=display, save=save)
        feature_set = hog_features
        if display == True: 
            plot_nxn(hog_images, fileName = f"{set_id}_hog", save=save)
        if benchmark is not None:
            benchmark_hog_features, benchmark_hog_image = display_hog(benchmark, set_id=set_id, display=display, save=save)
            benchmark_features = benchmark_hog_features[0]
            
    return images, feature_set, benchmark_features

In [35]:
## STEP 6 - Calculate Similarity
def similarity_procedure(images, benchmark, features, benchmark_features, set_id=None, params=None):
    """
    This function calculates the similarity between the input images and a benchmark image using a specified distance measure on both the image data and feature data.
    
    Parameters:
    images (numpy.ndarray or list of numpy.ndarray): A 2D numpy array or a list of 2D numpy arrays representing the image(s) to calculate similarity for.
    benchmark (numpy.ndarray): A 2D numpy array representing the benchmark image.
    features (list): A list of feature data generated from the input image(s) using histogram data or HoG.
    benchmark_features (numpy.ndarray): A 1D numpy array representing the feature data generated from the benchmark image.
    set_id (str): An optional string to use as a prefix for display and/or save filenames, must be either 'clean' or 'dirty'
    params (dict): An optional dictionary containing custom parameters to use for calculating similarity.

    Returns:
    tuple (numpy.ndarray or list of numpy.ndarray, list, list): 
    - images: A 2D numpy array or a list of 2D numpy arrays representing the image(s) passed as input.
    - image_distances: A list of distances calculated between the input images and the benchmark image using the specified distance measure on image data.
    - feature_distances: A list of distances calculated between the input images' features and the benchmark image's features using the specified distance measure on feature data.
    """
    image_distances = []
    feature_distances = []
    print("STEP 6: Performing the following tasks...")
    
    # DEFAULT PARAMETERS
    print("- setting default parameters: measure, display, save")
    measure = 'chi2'
    display = False
    save = False
    # UNPACK PARAMETERS
    if params is not None:
        print("- unpacking custom parameters")
        print("ERROR: Custom Parameters not set up for this step. Terminating Process")
        return images
    
    # calculate differences
    print(f"- calculating {measure} measure on image data")
    for image in images:
        distance = calculate_data_distance(image, benchmark, measure=measure)
        image_distances.append(distance)

    print(f"- calculating {measure} measure on feature data")
    
    for feature in features:     
        distance = calculate_data_distance(feature, benchmark_features, data_mode='feature', measure=measure)
        feature_distances.append(distance)

#     if measure == 'chi2':
#         print("- calculating chi2 measure")
#         image_distance = chi_squared_onimages(images, benchmark)
#         feature_distance = chi_squared_onfeatures(features, benchmark_features)

    return images, image_distances, feature_distances

In [ ]:
# # STEP 6 - Calculate Similarity
# def similarity_procedure(images, benchmark, features, benchmark_features, set_id=None, params=None):
#     distances = []
#     print("STEP 6: Performing the following tasks...")
    
#     # DEFAULT PARAMETERS
#     print("- setting default parameters: measure, display, save")
#     measure = 'chi2'
#     display = False
#     save = False
#     # UNPACK PARAMETERS
#     if params is not None:
#         print("- unpacking custom parameters")
#         print("ERROR: Custom Parameters not set up for this step. Terminating Process")
#         return images
    
# #     # generate benchmark features
# #     benchmark_features, benchmark_hog_images = display_hog(benchmark_set, set_id='dirty', display=display, save=save)

#     # calculate differences
#     if measure == 'linear':
#         print("- WARNING: linear calculation is not built")
#     if measure == 'chi2':
#         print("- calculating chi2 measure")
#         image_distance = chi_squared_onimages(images, benchmark)
#         feature_distance = chi_squared_onfeatures(features, benchmark_features)

# #       
# #         for hog_features in hog_features_set:
# #             # calculate the chi-squared distance between the histograms
# # #             distance = chi2(hog_features_benchmark[0], hog_features)
# #             distance = chisquare(hog_features_benchmark[0], hog_features)
# #             chi2_dists.append(distance)
# #             print(distance)

#     return images, image_distance, feature_distance

In [19]:
# def similarity_procedure(images, benchmark, set_id=None, measure=None, params=None, display=False, save=False):
#     # generate two hog descriptors
# reference_img = final_clean[6]
# dist_set=[]
# #for dirty_image in final_dirty:
# dis_set = chi_squared(final_dirty, reference_img)   
#     if measure == 'linear':
#         print("not built")
#         print("not built")
#     if measure == 'chi2':
#         chi2_dists = []
#         hog_features_benchmark, hog_images_benchmark = display_hog(benchmark, set_id='dirty', display=display, save=save)
#         hog_features_set, hog_set = display_hog(images, set_id='dirty', display=display, save=save)
# #         distance = chi2_distance(hog_features_benchmark[0], hog_features_set[0])
# #         print(distance)

#        distance = chi2_distance(hog_features_set, 
# #       
# #         for hog_features in hog_features_set:
# #             # calculate the chi-squared distance between the histograms
# # #             distance = chi2(hog_features_benchmark[0], hog_features)
# #             distance = chisquare(hog_features_benchmark[0], hog_features)
# #             chi2_dists.append(distance)
# #             print(distance)

#     return images

In [13]:
'''
look at section 3's remainder to see if it's useful for HOG gradients


remainder = np.array(target_sobel_max) - np.array(reference_sobel_max)  
#print(np.array(target_sobel_max).min(), np.array(target_sobel_max).max())  
remainder = np.clip(remainder, 0, 255)  
display_image(target_sobel_max)  
display_image(reference_sobel_max)  
display_image(remainder)  
'''

"\nlook at section 3's remainder to see if it's useful for HOG gradients\n\n\nremainder = np.array(target_sobel_max) - np.array(reference_sobel_max)  \n#print(np.array(target_sobel_max).min(), np.array(target_sobel_max).max())  \nremainder = np.clip(remainder, 0, 255)  \ndisplay_image(target_sobel_max)  \ndisplay_image(reference_sobel_max)  \ndisplay_image(remainder)  \n"

[GO TO TOP](#top)

<a id="feature-extraction"></a>
-----------------------------
## Feature Extraction

#### Shape & Contour Features

In [14]:
def calculate_moments(image):
    image = pil_to_numpy(image)
    # find indices of all white pixels in the image
    rows, cols = np.where(image == 255)
    x_points = (cols.min(), cols.max())
    y_points = (rows.min(), rows.max())
    start_and_end = (x_points, y_points)
#     print("Min(X,Y): (", cols.min(), ",", rows.min(),") and Max(X,Y): (",cols.max(), ",", rows.max(),")")

    # calculate mean x and y coordinates of white pixels
    x_bar = np.mean(cols)
    y_bar = np.mean(rows)
    
    # calculate second moments in x and y direction
    mu20 = np.mean((cols - x_bar)**2)
    mu02 = np.mean((rows - y_bar)**2)
    mu11 = np.mean((cols - x_bar)*(rows - y_bar))

    return x_bar, y_bar, mu20, mu02, mu11, start_and_end

def calculate_axes(image, display=False):
    image = pil_to_numpy(image)
    # calculate moments of the object
    x_bar, y_bar, mu20, mu02, mu11, start_and_end = calculate_moments(image)
    
    # calculate major and minor axes of the object
    major_axis = np.sqrt(((mu20 + mu02) + np.sqrt((mu20 - mu02)**2 + 4*mu11**2))/2)
    minor_axis = np.sqrt(((mu20 + mu02) - np.sqrt((mu20 - mu02)**2 + 4*mu11**2))/2)
    
    # calculate angle of rotation of the major axis
    if mu20 > mu02:
        angle = np.arctan((2*mu11)/(mu20 - mu02))/2
    else:
        angle = np.pi/2 - np.arctan((2*mu11)/np.abs(mu02 - mu20))/2
        
    # get the integer part and float part using divmod()
    extra_int, angle = divmod(angle, 1)
    
    if display:
#         print("\nx_bar: ", x_bar,
#               "\ny_bar: ", y_bar,
#               "\nmu20: ", mu20,
#               "\nmu02: ", mu02,
#               "\nmu11: ", mu11)
        print("\nmajor_axis: ", major_axis,
              "\nminor_axis: ", minor_axis,
              "\nangle: ", angle) 
    return major_axis, minor_axis, x_bar, y_bar, angle, start_and_end

def visualize_axes(image, x_bar, y_bar, x1, x2, x3, x4, y1, y2, y3, y4):
        
    # draw the major and minor axes on the image
    image_with_axes = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    cv2.line(image_with_axes, (x1, y1), (x2, y2), (0, 0, 255), thickness=2)
    cv2.line(image_with_axes, (x3, y3), (x4, y4), (0, 255, 0), thickness=2)
    cv2.circle(image_with_axes, (int(x_bar), int(y_bar)), 5, (255, 0, 0), -1)
    return image_with_axes

def major_minor_axes(image, display=False):
    image = pil_to_numpy(image)
    
    #calculate axes
    major_axis, minor_axis, x_bar, y_bar, angle, start_and_end = calculate_axes(image, display)
    
    #calculate line information
    x_points = start_and_end[0]
    y_points = start_and_end[1]
    
    # calculate the endpoints of the major axis
    cos_angle = np.cos(angle)
    sin_angle = np.sin(angle)
    
    major_slope = -sin_angle/cos_angle
    major_intercept = y_bar - major_slope*x_bar
    x1 = x_points[0]
    y1 = int(major_slope*x1 + major_intercept)
    x2 = x_points[1]
    y2 = int(major_slope*x2 + major_intercept)
        
    # calculate the endpoints of the minor axis
    minor_slope = cos_angle/sin_angle
    minor_intercept = y_bar - minor_slope*x_bar
    y3 = y_points[0]
    x3 = int((y3 - minor_intercept)/minor_slope)
    y4 = y_points[1]
    x4 = int((y4 - minor_intercept)/minor_slope)
        
    # calculate major and minor line lengths
    major_length = np.sqrt((x2-x1)**2 + (y2-y1)**2)
    minor_length = np.sqrt((x4-x3)**2 + (y4-y3)**2)
    
    # print out major and minor line lengths
    if display:
        print("major color: blue | axis: x-horizontal")
        print("major start: (", x1, ",", y1, ") | major end (", x2, ",", y2, ") | major slope: ", major_slope, "| major_intercept: ", major_intercept)
        
        print("minor color: green | axis: y-vertical")
        print("minor start: (", x3, ",", y3, ") | minor end (", x4, ",", y4, ") | minor slope: ", minor_slope, "| minor_intercept: ", minor_intercept)
    
        print("major_length: ", major_length, "\nminor_length: ", minor_length)
    
    if display:
        image_with_axes = visualize_axes(image, x_bar, y_bar, x1, x2, x3, x4, y1, y2, y3, y4)
        return image_with_axes, major_length, minor_length, angle
    else:
        axes_features = np.array([major_length, minor_length, angle])
        return axes_features

In [ ]:
def display_bounding_box(image, start_and_end, display=False):
    # calculate the coordinates of the bounding box corners
    x_points = start_and_end[0]
    y_points = start_and_end[1] 
    
    x1 = x_points[0]
    y1 = y_points[0]
    x2 = x_points[1]
    y2 = y_points[0]
    x3 = x_points[0]
    y3 = y_points[1]
    x4 = x_points[1]
    y4 = y_points[1]
    if display:
        print("bounding box corners: (", x1, ",", y1, "), (", x2, ",", y2, "), (", x3, ",", y3, "), (", x4, ",", y4, ")")
    
    # draw the bounding box as a set of contours using the display_contours function
#     contours = [[[x1, y1]], [[x2, y2]], [[x3, y3]], [[x4, y4]]]
#     display_bounding_contours(image, contours)
    # draw the bounding box on the image
    image_with_axes = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    cv2.rectangle(image_with_axes, (x1, y1), (x4, y4), (255, 0, 0), 2)

    # return the bounding box coordinates
#     return x1, y1, x2, y2, x3, y3, x4, y4
    return image_with_axes

def fill_bounding_box(image, start_and_end, fill_value=255):
    x1 = start_and_end[0][0]
    y1 = start_and_end[1][0]
    x2 = start_and_end[0][1]
    y2 = start_and_end[1][1]
    
    image = numpy_to_pil(image)
    draw = ImageDraw.Draw(image)
    draw.rectangle((x1,y1,x2,y2), fill=fill_value, outline=None)
#     image.show()

    return image.convert('L')

#### LBP Features

In [ ]:
def get_lbp_pixel(img, center, x, y):
    img = pil_to_numpy(img)
    value = 0
    if img[x][y] >= center:
        value = 1
    return value

def get_lbp_code(img, x, y):
    img = pil_to_numpy(img)
    center = img[x][y]
    code = 0
    code |= get_lbp_pixel(img, center, x-1, y-1) << 7
    code |= get_lbp_pixel(img, center, x-1, y) << 6
    code |= get_lbp_pixel(img, center, x-1, y+1) << 5
    code |= get_lbp_pixel(img, center, x, y+1) << 4
    code |= get_lbp_pixel(img, center, x+1, y+1) << 3
    code |= get_lbp_pixel(img, center, x+1, y) << 2
    code |= get_lbp_pixel(img, center, x+1, y-1) << 1
    code |= get_lbp_pixel(img, center, x, y-1) << 0
    return code

def get_lbp_feature(img, title=None, status=None, m=None, n=None, display=False, save=False):
    img = pil_to_numpy(img)
    height, width = img.shape
    lbp_img = np.zeros((height, width), np.uint8)
    for i in range(1, height-1):
        for j in range(1, width-1):
            lbp_img[i][j] = get_lbp_code(img, i, j)
    hist, _ = np.histogram(lbp_img.ravel(), bins=256, range=(0, 256))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)
    
    if display:
        # retrieve plot colors from main file
        %store -r plt_color
        %store -r label_color
        %store -r title_color

        with plt.rc_context({'axes.edgecolor':label_color, 'xtick.color':label_color, 'ytick.color':label_color}):
            if m is not None:
                fig = plt.figure(figsize=(m,n))
            else:
                fig = plt.figure(figsize=(8,2))

        # Plot the contour and the chain code
        _ = fig.add_subplot(1,2,1)
        plt.imshow(lbp_img, cmap='gray')
        plt.axis('off')
        plt.title(f'{status} LBP Image', color = title_color)

        _ = fig.add_subplot(1,2,2)
        plt.plot(hist, color= plt_color)
        plt.axis('off')
        plt.title(f'{status} LBP Histogram',  color = title_color)

        if save == True:
            _ = fig.savefig(f'{status}_LBP.png', dpi = 300, transparent=True, bbox_inches="tight")
        plt.show()

    return hist

#### Circularity

In [8]:
def calculate_circularity(image, return_as='value', display=False):
    area = np.sum(image > 0)
    perimeter = np.sum(feature.canny(image))
    circularity = 4 * np.pi * area / (perimeter ** 2) if perimeter > 0 else 0
    
    if display:
        print("Circularity = ", circularity)
    
    if return_as == 'value':
        return circularity
    if return_as == 'feature':
        circularity_feature = np.array([circularity])
        return circularity_feature
    else:
        raise TypeError("return_as must be either 'value' for a float return, or 'feature' for an array return.")

<a id="downsample-upsample"></a>
-----------------------------
### DOWNSAMPLE AND UPSAMPLE

In [8]:
def upscale_with_zeros(image, factor=2, display=False):
    image = pil_to_numpy(image)
    height, width = image.shape
    upscaled_image = np.zeros((height * factor, width * factor), dtype=image.dtype)
    upscaled_image[::factor, ::factor] = image
    
    if display:
        display_image(upscaled_image)
        
    upscaled_image = numpy_to_pil(upscaled_image)
    return upscaled_image

In [9]:
# def downsample(img):
#     return img.resize((img.width // 2, img.height // 2), Image.BICUBIC)

In [1]:
def downscale_gaussian(image, factor, n, display=False):
    def downscale_once(image, factor):
        image = numpy_to_pil(image)
        blurred_image, kernel = filter_gaussian2(image, sigma=1)
        return pil_to_numpy(blurred_image)[::factor, ::factor], kernel

    pyramid = []
    pyramid.append(pil_to_numpy(image))
    for _ in range(n):
        image, kernel = downscale_once(image, factor)
        pyramid.append(image)
  
    if display:
        print("NEW RESOLUTIONS")
        for i, arr in enumerate(pyramid):
            print(f"downsize #{i}", pyramid[i].shape)
        plot_nxn(pyramid)
        
    pyramid = numpy_to_pil(pyramid)
    return pyramid, kernel

In [ ]:
# def upscale_laplacian(pyramid, factor, display=False):
# #     def upscale_once(image, factor):
# #         image = numpy_to_pil(image)
# #         upscaled_image = image.resize((image.width * factor, image.height * factor), Image.BICUBIC)
# #         return pil_to_numpy(upscaled_image)
    
# #     laplacian_pyramid = [pyramid[-1]]
# #     for i in range(len(pyramid) - 2, -1, -1):
# #         upscaled_image = upscale_once(laplacian_pyramid[-1], factor)
# #         laplacian_level = pyramid[i] - upscaled_image
# #         laplacian_pyramid.append(laplacian_level)

# #     reconstructed_image = pyramid[0] + laplacian_pyramid[-1]
    
#     if display:
#         print("NEW RESOLUTIONS")
#         for i, arr in enumerate(pyramid):
#             print(f"upscaled #{i}", pyramid[i].shape)
#         plot_nxn(pyramid)
        
#     return pyramid, reconstructed_image


In [ ]:
'''
Your understanding of the Laplacian pyramid theory is mostly correct. However, there's a small correction. Instead of subtracting the original image from the Laplacian image, you actually subtract the upscaled and blurred image from the original image to obtain the Laplacian pyramid. The process can be summarized as:

    Create a Gaussian pyramid by successively downsampling and blurring the original image.
    For each level of the Gaussian pyramid:
    a. Upscale the image to the size of the next level up.
    b. Blur the upscaled image using the same Gaussian kernel.
    c. Subtract the blurred, upscaled image from the Gaussian image at the next level up to obtain the Laplacian image for that level.

To reconstruct the image from the Laplacian pyramid, you perform the inverse operations, starting from the smallest level and working your way up.
'''

In [7]:
# def build_gaussian_pyramid(image, n):
#     pyramid = [image]
#     for _ in range(n - 1):
#         image = image.filter(ImageFilter.GaussianBlur(radius=1))
#         pyramid.append(image)
#     return pyramid

# def build_laplacian_pyramid(gaussian_pyramid):
#     laplacian_pyramid = []
#     for i in range(len(gaussian_pyramid) - 1):
#         upsampled = upsample(gaussian_pyramid[i + 1])
#         laplacian = Image.fromarray(np.subtract(np.array(gaussian_pyramid[i]), np.array(upsampled)))
#         laplacian_pyramid.append(laplacian)
#     laplacian_pyramid.append(gaussian_pyramid[-1])
#     return laplacian_pyramid

# def upscale_laplacian(laplacian_pyramid, n):
#     result = laplacian_pyramid[-1]
#     for i in range(n - 1, -1, -1):
#         result = Image.fromarray(np.add(np.array(upsample(result)), np.array(laplacian_py


In [ ]:
# def upscale_laplacian(pyramid, factor, n):
#     def upscale_once(image, factor):
#         image = numpy_to_pil(image)
#         width, height = image.size
#         new_size = (int(width * factor), int(height * factor))
#         upscaled_image = image.resize(new_size, resample=Image.BICUBIC)
#         return pil_to_numpy(upscaled_image)

#     laplacian_pyramid = []
#     for i in range(n, 0, -1):
#         image = pyramid[i]
#         next_image = pyramid[i - 1] if i > 1 else None

#         # Upscale the image and subtract it from the next level in the pyramid
#         upscaled_image = upscale_once(image, factor)
#         if next_image is not None:
#             next_width, next_height = next_image.shape
#             upscaled_image = upscaled_image[:next_width, :next_height]
#             next_image = next_image - upscaled_image

#         # Compute the Laplacian
#         laplacian = convolve_image(image, kernel3x3_laplacian1)
#         laplacian = np.array(laplacian)
#         laplacian = laplacian - upscaled_image if next_image is not None else laplacian
#         laplacian_pyramid.append(laplacian)

#     # Reverse the Laplacian pyramid and add the original image at the top
#     laplacian_pyramid = laplacian_pyramid[::-1]
#     laplacian_pyramid.append(pyramid[0])

#     return laplacian_pyramid

In [ ]:
def upscale_laplacian(pyramid, factor, n):
    pyramid = pil_to_numpy(pyramid)
    
    laplacian_pyramid = []
    for i in range(n, 0, -1):
        image = pyramid[i]
        next_image = pyramid[i - 1] if i > 1 else None

        # Upscale the image and subtract it from the next level in the pyramid
        upscaled_image = upscale_with_zeros(image, factor)
        upscaled_image = pil_to_numpy(upscaled_image)
        print("Upscaled Image Shape1: ", upscaled_image.shape)
        if next_image is not None:
            next_width, next_height = next_image.shape
            upscaled_image = upscaled_image[:next_width, :next_height]
            next_image = next_image - upscaled_image

        # Compute the Laplacian
        laplacian = convolve_image(image, kernel3x3_laplacian1)
        laplacian = np.array(laplacian)
        print("Laplacian Shape: ", laplacian.shape)
        print("Upscaled Image Shape2: ", upscaled_image.shape)
        laplacian = laplacian - upscaled_image if next_image is not None else laplacian
        laplacian_pyramid.append(laplacian)

    # Reverse the Laplacian pyramid and add the original image at the top
    laplacian_pyramid = laplacian_pyramid[::-1]
    laplacian_pyramid.append(pyramid[0])

    return laplacian_pyramid


[GO TO TOP](#top)

In [2]:
# def generate_gaussian_pyramid(image, num_levels, downscale_factor):
#     pyramid = [image]
#     for i in range(1, num_levels):
#         image = image.filter(ImageFilter.GaussianBlur(downscale_factor))
#         image = image.resize((image.width // 2, image.height // 2), Image.ANTIALIAS)
#         pyramid.append(image)
#     return pyramid

# def generate_laplacian_pyramid(gaussian_pyramid):
#     laplacian_pyramid = []
#     for i in range(len(gaussian_pyramid) - 1):
#         size = (gaussian_pyramid[i].width, gaussian_pyramid[i].height)
#         upscale = gaussian_pyramid[i + 1].resize(size, Image.ANTIALIAS)
#         laplacian = ImageChops.subtract(gaussian_pyramid[i], upscale)
#         laplacian_pyramid.append(laplacian)
#     laplacian_pyramid.append(gaussian_pyramid[-1])
#     return laplacian_pyramid

# def blend_pyramids(laplacian_pyramid1, laplacian_pyramid2, mask_pyramid):
#     blended_pyramid = []
#     for i in range(len(laplacian_pyramid1)):
#         laplacian1 = np.asarray(laplacian_pyramid1[i])
#         laplacian2 = np.asarray(laplacian_pyramid2[i])
#         mask = np.asarray(mask_pyramid[i]) / 255.0
#         blended = laplacian1 * mask + laplacian2 * (1.0 - mask)
#         blended_pyramid.append(Image.fromarray(blended.astype(np.uint8)))
#     return blended_pyramid

# def reconstruct_image_from_laplacian_pyramid(laplacian_pyramid):
#     image = laplacian_pyramid[-1]
#     for i in range(len(laplacian_pyramid) - 2, -1, -1):
#         size = (laplacian_pyramid[i].width, laplacian_pyramid[i].height)
#         image = image.resize(size, Image.ANTIALIAS)
#         image = ImageChops.add(image, laplacian_pyramid[i])
#     return image

# def image_blending(img1, img2, mask, num_levels=4, downscale_factor=2, laplacian_kernel=None):
#     img1_gray = img1.convert("L")
#     img2_gray = img2.convert("L")
#     mask_gray = mask.convert("L")
    
#     img1_gaussian = generate_gaussian_pyramid(img1_gray, num_levels, downscale_factor)
#     img2_gaussian = generate_gaussian_pyramid(img2_gray, num_levels, downscale_factor)
#     mask_gaussian = generate_gaussian_pyramid(mask_gray, num_levels, downscale_factor)

#     img1_laplacian = generate_laplacian_pyramid(img1_gaussian)
#     img2_laplacian = generate_laplacian_pyramid(img2_gaussian)

#     blended_pyramid = blend_pyramids(img1_laplacian, img2_laplacian, mask_gaussian)
#     blended_image = reconstruct_image_from_laplacian_pyramid(blended_pyramid)

#     return blended_image


In [ ]:
# import numpy as np
# from PIL import Image, ImageFilter, ImageChops

# def generate_gaussian_pyramid(image, num_levels, downscale_factor):
#     pyramid = [image]
#     for i in range(1, num_levels):
#         image = image.filter(ImageFilter.GaussianBlur(downscale_factor))
#         image = image.resize((image.width // 2, image.height // 2), Image.ANTIALIAS)
#         pyramid.append(image)
#     return pyramid

# def apply_laplacian_kernel(image, kernel):
#     return image.filter(ImageFilter.Kernel(kernel.size, kernel.kernel, kernel.scale))

# def generate_laplacian_pyramid(gaussian_pyramid, laplacian_kernel):
#     laplacian_pyramid = []
#     for i in range(len(gaussian_pyramid) - 1):
#         size = (gaussian_pyramid[i].width, gaussian_pyramid[i].height)
#         upscale = gaussian_pyramid[i + 1].resize(size, Image.ANTIALIAS)
#         laplacian = ImageChops.subtract(gaussian_pyramid[i], upscale)
#         if laplacian_kernel:
#             laplacian = apply_laplacian_kernel(laplacian, laplacian_kernel)
#         laplacian_pyramid.append(laplacian)
#     laplacian_pyramid.append(gaussian_pyramid[-1])
#     return laplacian_pyramid

# def blend_pyramids(laplacian_pyramid1, laplacian_pyramid2, mask_pyramid):
#     blended_pyramid = []
#     for i in range(len(laplacian_pyramid1)):
#         laplacian1 = np.asarray(laplacian_pyramid1[i])
#         laplacian2 = np.asarray(laplacian_pyramid2[i])
#         mask = np.asarray(mask_pyramid[i]) / 255.0
#         blended = laplacian1 * mask + laplacian2 * (1.0 - mask)
#         blended_pyramid.append(Image.fromarray(blended.astype(np.uint8)))
#     return blended_pyramid

# def reconstruct_image_from_laplacian_pyramid(laplacian_pyramid):
#     image = laplacian_pyramid[-1]
#     for i in range(len(laplacian_pyramid) - 2, -1, -1):
#         size = (laplacian_pyramid[i].width, laplacian_pyramid[i].height)
#         image = image.resize(size, Image.ANTIALIAS)
#         image = ImageChops.add(image, laplacian_pyramid[i])
#     return image

# def image_blending(img1, img2, mask, num_levels=4, downscale_factor=2, laplacian_kernel=None):
#     img1_gray = img1.convert("L")
#     img2_gray = img2.convert("L")
#     mask_gray = mask.convert("L")
    
#     img1_gaussian = generate_gaussian_pyramid(img1_gray, num_levels, downscale_factor)
#     img2_gaussian = generate_gaussian_pyramid(img2_gray, num_levels, downscale_factor)
#     mask_gaussian = generate_gaussian_pyramid(mask_gray, num_levels, downscale_factor)

#     img1_laplacian = generate_laplacian_pyramid(img1_gaussian, laplacian_kernel)
#     img2_laplacian = generate_laplacian_pyramid(img2_gaussian, laplacian_kernel)

#     blended_pyramid = blend_pyramids(img1_laplacian,


In [ ]:
def generate_laplacian_pyramid_fun(gaussian_pyramid, laplacian_kernel):
    laplacian_pyramid = []
    for i in range(len(gaussian_pyramid) - 1):
        size = (gaussian_pyramid[i].width, gaussian_pyramid[i].height)
#         print(f"gaussian {i} size =", size)
#         print(f"gaussian {i+1} size =", pil_to_numpy(gaussian_pyramid[i+1]).shape)
#         upscale = gaussian_pyramid[i + 1].resize(size, Image.ANTIALIAS)
        upscale = upscale_with_zeros(gaussian_pyramid[i+1])
#         print(f"gaussian {i+1} upscale resize =", pil_to_numpy(upscale).shape)
        laplacian = ImageChops.subtract(gaussian_pyramid[i], upscale)
        
        if laplacian_kernel is not None:
            laplacian = convolve_image(laplacian, laplacian_kernel)

        laplacian_pyramid.append(laplacian)
        
    laplacian_pyramid.append(gaussian_pyramid[-1])
    
    return laplacian_pyramid

In [3]:
import numpy as np
from PIL import Image, ImageFilter, ImageChops

def generate_laplacian_pyramid2(gaussian_pyramid, laplacian_kernel=None):
    laplacian_pyramid = []
    for i in range(len(gaussian_pyramid) - 1):
#         size = (gaussian_pyramid[i].width, gaussian_pyramid[i].height)
#         print(f"gaussian {i} size =", size)
#         print(f"gaussian {i+1} size =", pil_to_numpy(gaussian_pyramid[i+1]).shape)
#         upscale = gaussian_pyramid[i + 1].resize(size, Image.ANTIALIAS)
        upscale = upscale_with_zeros(gaussian_pyramid[i+1])
#         print(f"gaussian {i+1} upscale resize =", pil_to_numpy(upscale).shape)
        laplacian = ImageChops.subtract(gaussian_pyramid[i], upscale)
        
        if laplacian_kernel is not None:
            laplacian = convolve_image(laplacian, laplacian_kernel)

        laplacian_pyramid.append(laplacian)
    laplacian_pyramid.append(gaussian_pyramid[-1])
    return laplacian_pyramid

def blend_pyramids2(laplacian_pyramid1, laplacian_pyramid2, mask_pyramid):
    blended_pyramid = []
    for i in range(len(laplacian_pyramid1)):
        laplacian1 = pil_to_numpy(laplacian_pyramid1[i])
        laplacian2 = pil_to_numpy(laplacian_pyramid2[i])
        mask = pil_to_numpy(mask_pyramid[i]) # / 255.0
        
        blended = (laplacian1 * mask) + (laplacian2 * (1 - mask))
        blended_pyramid.append(Image.fromarray(blended.astype(np.uint8)))
    return blended_pyramid

def reconstruct_image_from_laplacian_pyramid2(laplacian_pyramid):
    image = laplacian_pyramid[-1]
    for i in range(len(laplacian_pyramid) - 2, -1, -1):
        size = (laplacian_pyramid[i].width, laplacian_pyramid[i].height)
        image = image.resize(size, Image.LANCZOS)
        image = ImageChops.add(image, laplacian_pyramid[i])
    return image

def image_blending2(img1, img2, mask, num_levels=4, downscale_factor=2, laplacian_kernel=None):

    gaussian_pyramid = []
    laplacian_pyramid = []

    img1_gaussian = downscale_gaussian(img1, downscale_factor, num_levels)
    img2_gaussian = downscale_gaussian(img2, downscale_factor, num_levels)
    mask_gaussian = downscale_gaussian(mask, downscale_factor, num_levels)

    gaussian_pyramid += [img1_gaussian, img2_gaussian, mask_gaussian]

    img1_laplacian = generate_laplacian_pyramid2(img1_gaussian, laplacian_kernel)
    img2_laplacian = generate_laplacian_pyramid2(img2_gaussian, laplacian_kernel)

    laplacian_pyramid += [img1_laplacian, img2_laplacian]

    blended_pyramid = blend_pyramids2(img1_laplacian, img2_laplacian, mask_gaussian)
    blended_image = reconstruct_image_from_laplacian_pyramid2(blended_pyramid)

    return blended_image, blended_pyramid, gaussian_pyramid, laplacian_pyramid



In [ ]:
import numpy as np
from PIL import Image, ImageFilter, ImageChops

def apply_kernel(image, kernel):
    return image.filter(kernel)

def generate_gaussian_pyramid(image, num_levels, downscale_factor):
    pyramid = [image]
    for i in range(1, num_levels):
        image = image.filter(ImageFilter.GaussianBlur(downscale_factor))
        image = image.resize((image.width // 2, image.height // 2), Image.ANTIALIAS)
        pyramid.append(image)
    return pyramid

def generate_laplacian_pyramid(gaussian_pyramid, laplacian_kernel):
    laplacian_pyramid = []
    for i in range(len(gaussian_pyramid) - 1):
        size = (gaussian_pyramid[i].width, gaussian_pyramid[i].height)
        upscale = gaussian_pyramid[i + 1].resize(size, Image.ANTIALIAS)
        laplacian = ImageChops.subtract(gaussian_pyramid[i], upscale)
        
        if laplacian_kernel is not None:
            laplacian = apply_kernel(laplacian, laplacian_kernel)

        laplacian_pyramid.append(laplacian)
    laplacian_pyramid.append(gaussian_pyramid[-1])
    return laplacian_pyramid

def blend_pyramids(laplacian_pyramid1, laplacian_pyramid2, mask_pyramid):
    blended_pyramid = []
    for i in range(len(laplacian_pyramid1)):
        laplacian1 = np.asarray(laplacian_pyramid1[i])
        laplacian2 = np.asarray(laplacian_pyramid2[i])
        mask = np.asarray(mask_pyramid[i]) / 255.0
        blended = laplacian1 * mask + laplacian2 * (1.0 - mask)
        blended_pyramid.append(Image.fromarray(blended.astype(np.uint8)))
    return blended_pyramid

def reconstruct_image_from_laplacian_pyramid(laplacian_pyramid):
    image = laplacian_pyramid[-1]
    for i in range(len(laplacian_pyramid) - 2, -1, -1):
        size = (laplacian_pyramid[i].width, laplacian_pyramid[i].height)
        image = image.resize(size, Image.ANTIALIAS)
        image = ImageChops.add(image, laplacian_pyramid[i])
    return image

def image_blending(img1, img2, mask, num_levels=4, downscale_factor=2, laplacian_kernel=None):
    
    img1_gray = img1.convert("L")
    img2_gray = img2.convert("L")
    mask_gray = mask.convert("L")

    img1_gaussian = generate_gaussian_pyramid(img1, num_levels, downscale_factor)
    img2_gaussian = generate_gaussian_pyramid(img2, num_levels, downscale_factor)
    mask_gaussian = generate_gaussian_pyramid(mask, num_levels, downscale_factor)

    img1_laplacian = generate_laplacian_pyramid(img1_gaussian, laplacian_kernel)
    img2_laplacian = generate_laplacian_pyramid(img2_gaussian, laplacian_kernel)

    blended_pyramid = blend_pyramids(img1_laplacian, img2_laplacian, mask_gaussian)
    blended_image = reconstruct_image_from_laplacian_pyramid(blended_pyramid)

    return blended_image, img1_laplacian, img2_laplacian
